# код

In [1]:
G1 = {(1,1):1, (1,2):2, (1,3):3, (1,4):4, 
      (2,1):2, (2,2):1, (2,3):4, (2,4):3, 
      (3,1):3, (3,2):4, (3,3):1, (3,4):2, 
      (4,1):4, (4,2):3, (4,3):2, (4,4):1 }
G2 = [[1,2,3,4],
      [2,1,4,3],
      [3,4,1,2],
      [4,3,2,1]]


In [15]:
def toMap(G):
    if type(G)==map: return G
    G1 = {}
    for i in range(len(G)):
        for j in range(len(G[i])):
            if type(G[i][j])==int and G[i][j]!=0:
                G1[(i+1,j+1)]=G[i][j]
    return G1

def ijmax(G):
    imax=0; jmax=0
    for k in G:
        imax = max(imax,k[0])
        jmax = max(jmax,k[1])
    return imax, jmax

def toArr(G):
    if type(G)==list: return G
    imax, jmax = ijmax(G)
    G1 = [[0 for j in range(jmax)] for i in range(imax)]
    for k in G:
        i=k[0]-1
        j=k[1]-1
        G1[i][j]=G[k]
    return G1

In [16]:
toArr(toMap(G2))

[[1, 2, 3, 4], [2, 1, 4, 3], [3, 4, 1, 2], [4, 3, 2, 1]]

In [17]:
max(ijmax(G1))

4

In [18]:
def printG(G):
    for line in toArr(G):
        print(*line)

In [19]:
DEBUG = True
VERBOSE = True
def check(G):
    r = max(ijmax(G))
    for a in range(1,r+1):
        for b in range(1,r+1):
            for c in range(1,r+1):
                if (a,b)in G and (b,c)in G:
                    ab=G[(a,b)]; bc = G[(b,c)]
                    if (ab,c)in G and (a,bc) in G:
                        if G[(ab,c)]!=G[(a,bc)]:
                            if VERBOSE: print('abc!=abc',a,b,c)
                            return False
    return True
def fillId(G,N):
    G[(1,1)]=1
    for i in range(2,N+1):
        G[(1,i)]=i
        G[(i,1)]=i
    return G
def fillInv(G,N):
    '''делает так, чтоб в каждой строке/столбце была ровно одна единица'''
    for i in range(1,N+1):
        c=0;p=0
        for j in range(1,N+1):
            if (i,j) not in G:
                c+=1
                p=j
            elif G[(i,j)]==1:
                c+=1
                p=0
                break
        if c==0:
            if DEBUG: print(i,'* x != 1 for Any x')
            return False
        if p!=0 and c==1:
            G[(i,p)]=1
    for i in range(1,N+1):
        c=0;p=0
        for j in range(1,N+1):
            if (j,i) not in G:
                c+=1
                p=j
            elif G[(j,i)]==1:
                c+=1
                p=0
                break
        if c==0:
            if DEBUG: print(i,'* x != 1 for Any x')
            return False
        if p!=0 and c==1:
            G[(p,i)]=1
    return True
def fill(G,N):
    #fillInv(G,N)
    changed = True
    while changed:
        changed = False
        for i in range(1, N+1):
            for j in range(1,N+1):
                if (i,j) not in G:
                    candidates = set()
                    for k in range(1,N+1):
                        G[(i,j)]=k
                        if VERBOSE: print('assume',i,j,k)
                        #fillInv(G,N)
                        if check(G):
                            candidates.add(k)
                    del G[(i,j)]
                    if len(candidates)==0:
                        if DEBUG: print(i,'*',j,'has no candidates')
                        return False
                    elif len(candidates)==1:
                        if VERBOSE: print('setup ',i,j,next(iter(candidates)))
                        G[(i,j)]=next(iter(candidates))
                        changed = True
    return True

In [20]:
next(iter({1}))

1

In [21]:
G4 = fillId({},4)
for line in toArr(G4):
    print(*line)

1 2 3 4
2 0 0 0
3 0 0 0
4 0 0 0


In [22]:
import itertools
import copy

def genId(N):
    '''генерирует список групп с расставленными по всякому единицами'''
    Gs = []
    global DEBUG
    DEBUG = False
    for p in itertools.permutations(range(2,N+1)):
        G = fillId({},N)
        for i in range(2,N+1):
            G[(i,p[i-2])]=1
        if fill(G,N):
#            c = 0
#            if (2,2) in G:
#                for k in range(1,N+1):
#                    G1 = copy.copy(G)
#                    G1[(2,3)] = k
#                    if fill(G1,N): c+=1
#            else:
#                for k in range(1,N+1):
#                    G1 = copy.copy(G)
#                    G1[(2,2)] = k
#                    if fill(G1,N): c+=1
#            if c>0:
                Gs.append(G)
    DEBUG = True
    return Gs


In [23]:
def genFill(G,N,ii=1,sets=None):
    '''начиная с ii-й строки находит пустоую клетку и начинает туда подставлять значения, и рекурсивно запускать себя
    а когда не находит пустую клетку - выводит всю группу'''
    global DEBUG
    DEBUG = False
    if sets==None: sets = []
    isClean = True # в нем нет дыр
    
    i1=0; j1=0
    for i in range(ii,N+1):
        for j in range(1,N+1):
            if (i,j) not in G:
                #print(sets,i,j)
                isClean = False
                i1=i; j1=j
                break
        if not isClean: break
    #print(sets,ii,i1,j1)
    if isClean:
        print(sets)
        for line in toArr(G):
            print(*line)
        print()
        return 1
    else:
        count = 0
        #print(i1,j1)
        for k in range(1,N+1):
            G1 = copy.copy(G)
            G1[(i1,j1)]=k
            if fill(G1,N):
                count += genFill(G1,N,i1,[*sets,(i1,j1,k)])
                DEBUG = False
        DEBUG = True
        return count

# left inverse, left 1

In [24]:
G4 = {}
G4[(1,1)]=1

G4[(1,2)]=2
G4[(1,3)]=3
G4[(1,4)]=4
G4[(1,5)]=5
G4[(1,6)]=6

G4[(2,3)]=1
G4[(3,4)]=1
G4[(2,2)]=5

VERBOSE = False
fill(G4,6)
printG(G4)
print()

#G4[(2,2)]=6
#G4[(2,4)]=3
#G4[(4,4)]=2
#VERBOSE = False
#fill(G4,6)
#printG(G4)

1 2 3 4 5 6
4 5 1 3 6 2
3 6 4 1 2 5
4 5 1 3 6 2
3 6 4 1 2 5
1 2 3 4 5 6



In [25]:
G5 = copy.copy(G4)
#G5[(1,3)]=0
fill(G5,8)
printG(G5)

1 2 3 4 5 6
4 5 1 3 6 2
3 6 4 1 2 5
4 5 1 3 6 2
3 6 4 1 2 5
1 2 3 4 5 6


# left-right

In [9]:
G4 = {}
G4[(1,1)]=1

G4[(1,2)]=2
G4[(1,3)]=3
G4[(1,4)]=4
G4[(1,5)]=5
G4[(1,6)]=6
G4[(1,7)]=7
G4[(1,8)]=8
G4[(1,9)]=9
G4[(1,10)]=10

G4[(2,3)]=1
G4[(3,4)]=1
#G4[(4,5)]=1
#G4[(5,6)]=1
#G4[(6,7)]=1
printG(G4)
print()

VERBOSE = False
fill(G4,10)
printG(G4)
print()

G4[(3,2)]=5
#G4[(4,4)]=2
VERBOSE = True
fill(G4,10)
printG(G4)

NameError: name 'toArr' is not defined

# 2

In [26]:
r = 2
G4 = fillId({},r)
fillInv(G4,r)
fill(G4,r)
for line in toArr(G4):
    print(*line)

1 2
2 1


# 3

In [27]:
r = 3
G4 = fillId({},r)
G4[(2,2)]=3
fillInv(G4,r)
fill(G4,r)
fillInv(G4,r)
for line in toArr(G4):
    print(*line)

1 2 3
2 3 1
3 1 2


#  4

In [28]:
r = 4
G4 = fillId({},r)
G4[(2,2)]=1
G4[(2,3)]=4
G4[(3,3)]=1

fillInv(G4,r)
fill(G4,r)
fillInv(G4,r)
for line in toArr(G4):
    print(*line)

1 2 3 4
2 1 4 3
3 4 1 2
4 3 2 1


In [29]:
r = 4
G4 = fillId({},r)
G4[(2,2)]=1
G4[(2,3)]=4
G4[(3,3)]=2

fillInv(G4,r)
fill(G4,r)
fillInv(G4,r)
for line in toArr(G4):
    print(*line)

1 2 3 4
2 1 4 3
3 4 2 1
4 3 1 2


In [30]:
r = 4
G4 = fillId({},r)
G4[(2,2)]=3
G4[(2,3)]=4

fillInv(G4,r)
fill(G4,r)
fillInv(G4,r)
for line in toArr(G4):
    print(*line)

1 2 3 4
2 3 4 1
3 4 1 2
4 1 2 3


In [31]:
r = 4
G4 = fillId({},r)
G4[(2,2)]=4
G4[(2,3)]=1

fillInv(G4,r)
fill(G4,r)
fillInv(G4,r)
for line in toArr(G4):
    print(*line)

1 2 3 4
2 4 1 3
3 1 4 2
4 3 2 1


#  5

In [32]:
G5s = genId(5)
len(G5s)

10

In [33]:
import copy
DEBUG = True

G = copy.copy(G5s[5])
G[(2,2)]=4
fill(G,5)
for line in toArr(G):
    print(*line)

1 2 3 4 5
2 4 1 5 3
3 1 5 2 4
4 5 2 3 1
5 3 4 1 2


In [34]:
G = copy.copy(G5s[5])
G[(2,2)]=5
fill(G,5)
for line in toArr(G):
    print(*line)

1 2 3 4 5
2 5 1 3 4
3 1 4 5 2
4 3 5 2 1
5 4 2 1 3


In [35]:
G = copy.copy(G5s[7])
G[(2,2)]=3
fill(G,5)
for line in toArr(G):
    print(*line)

1 2 3 4 5
2 3 5 1 4
3 5 4 2 1
4 1 2 5 3
5 4 1 3 2


In [36]:
G = copy.copy(G5s[7])
G[(2,2)]=5
fill(G,5)
for line in toArr(G):
    print(*line)

1 2 3 4 5
2 5 4 1 3
3 4 2 5 1
4 1 5 3 2
5 3 1 2 4


In [37]:
G = copy.copy(G5s[9])
G[(2,2)]=3
fill(G,5)
for line in toArr(G):
    print(*line)

1 2 3 4 5
2 3 4 5 1
3 4 5 1 2
4 5 1 2 3
5 1 2 3 4


In [24]:
G = copy.copy(G5s[9])
G[(2,2)]=4
fill(G,5)
for line in toArr(G):
    print(*line)

1 2 3 4 5
2 4 5 3 1
3 5 2 1 4
4 3 1 5 2
5 1 4 2 3


#  6

In [25]:
G5s = genId(6)
len(G5s)

26

In [26]:
c = 0
for n in range(len(G5s)):
    c1 = genFill(G5s[n],6)
    c+=c1
    print('--',c1)
print('----',c)

-- 0
[(2, 3, 5)]
1 2 3 4 5 6
2 1 5 6 3 4
3 6 1 5 4 2
4 5 6 1 2 3
5 4 2 3 6 1
6 3 4 2 1 5

[(2, 3, 6)]
1 2 3 4 5 6
2 1 6 5 4 3
3 5 1 6 2 4
4 6 5 1 3 2
5 3 4 2 6 1
6 4 2 3 1 5

-- 2
[(2, 3, 4)]
1 2 3 4 5 6
2 1 4 3 6 5
3 5 1 6 2 4
4 6 2 5 1 3
5 3 6 1 4 2
6 4 5 2 3 1

[(2, 3, 5)]
1 2 3 4 5 6
2 1 5 6 3 4
3 4 1 2 6 5
4 3 6 5 1 2
5 6 2 1 4 3
6 5 4 3 2 1

-- 2
[(2, 3, 4)]
1 2 3 4 5 6
2 1 4 3 6 5
3 6 1 5 4 2
4 5 2 6 3 1
5 4 6 2 1 3
6 3 5 1 2 4

[(2, 3, 6)]
1 2 3 4 5 6
2 1 6 5 4 3
3 4 1 2 6 5
4 3 5 6 2 1
5 6 4 3 1 2
6 5 2 1 3 4

-- 2
[(2, 3, 5)]
1 2 3 4 5 6
2 1 5 6 3 4
3 6 4 1 2 5
4 5 1 3 6 2
5 4 6 2 1 3
6 3 2 5 4 1

[(2, 3, 6)]
1 2 3 4 5 6
2 1 6 5 4 3
3 5 4 1 6 2
4 6 1 3 2 5
5 3 2 6 1 4
6 4 5 2 3 1

-- 2
[(2, 3, 5), (3, 3, 4)]
1 2 3 4 5 6
2 1 5 6 3 4
3 5 4 1 6 2
4 6 1 3 2 5
5 3 6 2 4 1
6 4 2 5 1 3

[(2, 3, 5), (3, 3, 6)]
1 2 3 4 5 6
2 1 5 6 3 4
3 5 6 1 4 2
4 6 1 5 2 3
5 3 4 2 6 1
6 4 2 3 1 5

[(2, 3, 6), (3, 3, 4)]
1 2 3 4 5 6
2 1 6 5 4 3
3 6 4 1 2 5
4 5 1 3 6 2
5 4 2 6 3 1
6 3 5 2 1 4

[(2, 3,

#  7

In [27]:
G5s = genId(7)
len(G5s)
#executed in 2.44s, finished 17:30:58 2022-12-12

76

In [28]:
c = 0
for n in range(len(G5s)):
    c1 = genFill(G5s[n],7)
    c+=c1
    print(n,': --',c1)
print('----',c)
#executed in 57.8s, finished 17:31:57 2022-12-12

0 : -- 0
1 : -- 0
2 : -- 0
3 : -- 0
4 : -- 0
5 : -- 0
6 : -- 0
7 : -- 0
8 : -- 0
9 : -- 0
10 : -- 0
11 : -- 0
12 : -- 0
13 : -- 0
14 : -- 0
15 : -- 0
16 : -- 0
17 : -- 0
18 : -- 0
19 : -- 0
20 : -- 0
21 : -- 0
22 : -- 0
23 : -- 0
24 : -- 0
25 : -- 0
26 : -- 0
27 : -- 0
28 : -- 0
29 : -- 0
30 : -- 0
[(2, 2, 4), (2, 4, 6)]
1 2 3 4 5 6 7
2 4 1 6 3 7 5
3 1 5 2 7 4 6
4 6 2 7 1 5 3
5 3 7 1 6 2 4
6 7 4 5 2 3 1
7 5 6 3 4 1 2

[(2, 2, 4), (2, 4, 7)]
1 2 3 4 5 6 7
2 4 1 7 3 5 6
3 1 5 2 6 7 4
4 7 2 6 1 3 5
5 3 6 1 7 4 2
6 5 7 3 4 2 1
7 6 4 5 2 1 3

[(2, 2, 5), (2, 5, 6)]
1 2 3 4 5 6 7
2 5 1 3 6 7 4
3 1 4 7 2 5 6
4 3 7 6 1 2 5
5 6 2 1 7 4 3
6 7 5 2 4 3 1
7 4 6 5 3 1 2

[(2, 2, 5), (2, 5, 7)]
1 2 3 4 5 6 7
2 5 1 3 7 4 6
3 1 4 6 2 7 5
4 3 6 7 1 5 2
5 7 2 1 6 3 4
6 4 7 5 3 2 1
7 6 5 2 4 1 3

[(2, 2, 6), (2, 4, 5)]
1 2 3 4 5 6 7
2 6 1 5 7 4 3
3 1 7 6 4 2 5
4 5 6 3 1 7 2
5 7 4 1 2 3 6
6 4 2 7 3 5 1
7 3 5 2 6 1 4

[(2, 2, 6), (2, 4, 7)]
1 2 3 4 5 6 7
2 6 1 7 4 5 3
3 1 7 5 6 2 4
4 7 5 2 1 3 6
5 4 6 1 3 7

[(2, 2, 7), (2, 3, 6)]
1 2 3 4 5 6 7
2 7 6 5 1 4 3
3 6 5 2 7 1 4
4 5 2 3 6 7 1
5 1 7 6 4 3 2
6 4 1 7 3 2 5
7 3 4 1 2 5 6

53 : -- 8
54 : -- 0
[(2, 2, 3), (2, 3, 4)]
1 2 3 4 5 6 7
2 3 4 6 1 7 5
3 4 6 7 2 5 1
4 6 7 5 3 1 2
5 1 2 3 7 4 6
6 7 5 1 4 2 3
7 5 1 2 6 3 4

[(2, 2, 3), (2, 3, 6)]
1 2 3 4 5 6 7
2 3 6 7 1 4 5
3 6 4 5 2 7 1
4 7 5 2 6 1 3
5 1 2 6 7 3 4
6 4 7 1 3 5 2
7 5 1 3 4 2 6

[(2, 2, 4), (2, 3, 6)]
1 2 3 4 5 6 7
2 4 6 7 1 5 3
3 6 2 5 7 4 1
4 7 5 3 2 1 6
5 1 7 2 6 3 4
6 5 4 1 3 7 2
7 3 1 6 4 2 5

[(2, 2, 4), (2, 3, 7)]
1 2 3 4 5 6 7
2 4 7 3 1 5 6
3 7 5 6 4 2 1
4 3 6 7 2 1 5
5 1 4 2 6 7 3
6 5 2 1 7 3 4
7 6 1 5 3 4 2

[(2, 2, 6), (2, 3, 4)]
1 2 3 4 5 6 7
2 6 4 5 1 7 3
3 4 2 6 7 5 1
4 5 6 7 3 1 2
5 1 7 3 4 2 6
6 7 5 1 2 3 4
7 3 1 2 6 4 5

[(2, 2, 6), (2, 3, 7)]
1 2 3 4 5 6 7
2 6 7 5 1 3 4
3 7 5 2 6 4 1
4 5 2 3 7 1 6
5 1 6 7 4 2 3
6 3 4 1 2 7 5
7 4 1 6 3 5 2

[(2, 2, 7), (2, 4, 3)]
1 2 3 4 5 6 7
2 7 5 3 1 4 6
3 5 6 7 4 2 1
4 3 7 2 6 1 5
5 1 4 6 3 7 2
6 4 2 1 7 5 3
7 6 1 5 2 3 4

[(2,

#  8

In [136]:
if False:
    G8s = genId(8)
    len(G5s)
    #executed in 16.7s, finished 17:32:46 2022-12-12

In [137]:
if False:
    c = 0
    for n in range(len(G8s)):
        c1 = genFill(G8s[n],8)
        c+=c1
        print(n,': --',c1)
    print('----',c)
    #executed in 17m 1s, finished 17:50:06 2022-12-12
    #---- 2760

# еще код

In [1]:
import itertools
import copy

class Group:
    __slots__ = ['N','_mul','_dec','_inv']; # multiplication, decomposition
    def __init__(self,N,perm=None):
        """можно инициализировать текстовой таблицей
        можно инициализировать порядком и, опционально, списком единиц
        """
        if type(N) is str and perm==None:
            self._mul = [[int(x) for x in  l.strip().split(' ')] for l in N.split('\n') if l.strip()!='']
            self.N = len(self._mul)
            N = self.N
            for i in range(N):
                assert len(self._mul)==N
            assert self.check()
            
            self._dec = [set() for i in range(N)]
            self._inv = [0 for i in range(N)]
            for i in range(1,N+1):
                for j in range(1,N+1):
                    self.setup(i,j,self._mul[i-1][j-1])
        else:
            self.N = N
            self._mul = [[0 for i in range(N)] for j in range(N)]
            self._dec = [set() for i in range(N)]
            self._inv = [0 for i in range(N)]

            self.setup( 1,1 , 1 )
            for i in range(2,N+1):
                self.setup( 1,i , i )
                self.setup( i,1 , i )

            if perm!=None:
                assert len(perm)==N-1 and min(perm)==2 and max(perm)==N
                for i in range(2,N+1):
                    self.setup(i,perm[i-2],1)
                
    def __repr__(self):
        return '\n'.join([' '.join([str(x) for x in self._mul[i]]) for i in range(self.N)]) if self.N<10 else \
            '\n'.join([' '.join([str(x).zfill(2) for x in self._mul[i]]) for i in range(self.N)]) if self.N<100 else \
            '\n'.join([' '.join([str(x).zfill(3) for x in self._mul[i]]) for i in range(self.N)])
            
    def setup(self, a,b , c):
        "проверку на ассоциативность не делает"
        self._mul[a-1][b-1]=c
        if c!=0:
            self._dec[c-1].add((a,b))
        else:
            self._dec[c-1].discard((a,b))
        if c==1:
            self._inv[a-1] = b
            self._inv[b-1] = a
        
    def __call__(self,a,b,c=None):
        if c is not None : return self(self(a,b),c)
        if type(a) is not int or type(b) is not int:
            # т.е. значения могут быть множествами, 
            # todo привести в соответствие с этим весь оставшийся код
            if type(a) is int: a=[a]
            if type(b) is int: b=[b]
            r = set()
            for aa in a:
                for bb in b:
                    r.add(self(aa,bb))
            return r
        else:
            return self._mul[a-1][b-1]
               
    
    def inv(self,a):
        return self._inv[a-1]
    
    def decomp(self,x):
        return self._dec[x-1]
    
    def check(self):
        N = self.N
        for a in range(1,N+1):
            for b in range(1,N+1):
                for c in range(1,N+1):
                    if self(a,b) !=0 and self(b,c) !=0:
                        ab=self(a,b); bc = self(b,c)
                        if self(ab,c) !=0 and self(a,bc) !=0:
                            if self(ab,c)!=self(a,bc):
                                return False
        return True

    def fill(self):
        #fillInv(G,N)
        changed = True
        while changed:
            changed = False
            for i in range(1, N+1):
                for j in range(1,N+1):
                    if self(i,j) ==0:
                        candidates = set()
                        for k in range(1,N+1):
                            self.setup( i,j , k )
                            #fillInv(G,N)
                            if self.check(): # оптимизировано в fastCheck ниже
                                candidates.add(k)
                        self.setup( i,j , 0 )
                        if len(candidates)==0:
                            if DEBUG: print(i,'*',j,'has no candidates')
                            return False
                        elif len(candidates)==1:
                            self.setup( i,j , next(iter(candidates)) )
                            changed = True
        return True


In [2]:
Group("""
   01 02 03 04 05 06 07 08 09 10
   02 01 08 07 10 09 04 03 06 05
   03 07 01 10 09 08 02 06 05 04
   04 08 09 01 07 10 05 02 03 06
   05 09 10 08 01 07 06 04 02 03
   06 10 07 09 08 01 03 05 04 02
   07 03 06 02 04 05 10 01 08 09
   08 04 02 05 06 03 01 09 10 07
   09 05 04 06 03 02 08 10 07 01
   10 06 05 03 02 04 09 07 01 08""")


01 02 03 04 05 06 07 08 09 10
02 01 08 07 10 09 04 03 06 05
03 07 01 10 09 08 02 06 05 04
04 08 09 01 07 10 05 02 03 06
05 09 10 08 01 07 06 04 02 03
06 10 07 09 08 01 03 05 04 02
07 03 06 02 04 05 10 01 08 09
08 04 02 05 06 03 01 09 10 07
09 05 04 06 03 02 08 10 07 01
10 06 05 03 02 04 09 07 01 08

In [3]:
DEBUG = True
VERBOSE = False
def fastCheck(G,a,b,c,conc):#,hints):
    """проверяет всё необходимое после установки a*b=c
    следствия помещает в словарь conc(lusions)
    """
    if VERBOSE: print('check',a,b,c)
    assert G(a,b)==c
    N = G.N
    
    def add_conc(a,b,c,mes):
        if VERBOSE: print('conc',mes,a,b,c)
        nonlocal conc
        #nonlocal hints
        if (a,b) in conc:
            if conc[(a,b)]!=c:
                if DEBUG: print('fail add_conc',a,b,c)
                return False
        #elif (a,b) in hints:
        #    if c not in hints[(a,b)]:
        #        if DEBUG: print('add_conc hints',a,b,c)
        #        return False
        #    hints.remove((a,b))
        #    conc[(a,b)]=c
        else:
            conc[(a,b)]=c
        return True
    #def add_hints(a,b,h):
    #    return True
    #    nonlocal hints
    #    nonlocal conc
    #    if len(h)==1:
    #        add_conc(a,b,next(iter(h)))
    #    elif (a,b) in hints:
    #        nh = h&hints[(a,b)]
    #        if len(nh)==0:
    #            if DEBUG: print('add_hints hints',a,b,h)
    #            return False
    #        elif len(nh)==1:
    #            conc[(a,b)]=next(iter(nh))
    #            hints.remove((a,b)) 
    #        else:
    #            hints[(a,b)]=nh
    #    elif (a,b) in conc:
    #        print('странно')
    #        if conc[(a,b)] not in h:
    #            if DEBUG: print('add_hints conc',a,b,h)
    #            return False
    #    else:
    #        hints[(a,b)]=h
    #    return True
            
    # ================================
    # x*c == (x*a)*b
    for x in range(1,N+1):
        xa = G(x,a)
        xc = G(x,c)
        if xa!=0 and xc!=0:
            if G(xa,b)!=0:
                if G(xa,b)!=xc:
                    if DEBUG: print('G(xa,b)!=xc',x,a,b,x,c)
                    return False
            else:
                #if (xa,b) in conc and conc[(xa,b)]!=xc:
                #    if DEBUG: print('conc[(xa,b)]!=xc',x,a,b,x,c)
                #    return False
                if not add_conc(xa,b, xc ,'xa,b, xc'): return False
        elif xa!=0:
            if G(xa,b)!=0:
                if not add_conc(x,c, G(xa,b) ,'x,c, G(xa,b)'): return False
        elif xc!=0:
            if G.inv(b)!=0 and G(xc,G.inv(b))!=0:
                if not add_conc(x,a, G(xc,G.inv(b)) , 'x,a, G(xc,G.inv(b))'): return False
            #h = set()
            #for (k,bb) in G.decomp(xc):
            #    if bb==b:
            #        h.add(k)
            #if not add_hints(x,a , h): return False
                
    # c*z == a*(b*z)
    for z in range(1,N+1):
        bz = G(b,z)
        cz = G(c,z)
        if bz!=0 and cz!=0:
            if G(a,bz)!=0:
                if G(a,bz)!=cz:
                    if DEBUG: print('G(a,bz)!=cz',a,b,z,c,z)
                    return False
            else:
                #if (a,bz) in conc and conc[(a,bz)]!=cz:
                #    if DEBUG: print('conc[(a,bz)]!=cz',a,b,z,c,z)
                #    return False
                if not add_conc(a,bz, cz ,'a,bz, cz'): return False
        elif bz!=0:
            if G(a,bz)!=0:
                if not add_conc(c,z, G(a,bz) ,'c,z, G(a,bz)'): return False
        elif cz!=0:
            if G.inv(a)!=0 and G(G.inv(a),cz)!=0:
                if not add_conc(b,z, G(G.inv(a),cz) ,'b,z, G(G.inv(a),cz)'): return False
            #h = set()
            #for (aa,k) in G.decomp(cz):
            #    if aa==a:
            #        h.add(k)
            #if not add_hints(b,z , h): return False
                
    # b==y*z => (a*y)*z = c
    for (y,z) in G.decomp(b):
        ay = G(a,y)
        if ay!=0:
            ayz = G(ay,z)
            if ayz!=0:
                if ayz!=c:
                    if DEBUG: print('ayz!=c',a,y,z,c)
                    return False
            else:
                if not add_conc(ay,z , c ,'ay,z , c'): return False
        else:
            if G.inv(z)!=0 and G(c,G.inv(z))!=0:
                if not add_conc(a,y, G(c,G.inv(z)) ,'a,y, G(c,G.inv(z))'): return False
            #h = set()
            #for (k,zz) in G.decomp(c):
            #    if zz==z:
            #        h.add(k)
            #if not add_hints(a,y , h ): return False

    # a==x*y => x*(y*b) = c
    for (x,y) in G.decomp(a):
        yb = G(y,b)
        if yb!=0:
            xyb = G(x,yb)
            if xyb!=0:
                if xyb!=c:
                    if DEBUG: print('xyb!=c',x,y,b,c)
                    return False
            else:
                if not add_conc(x,yb , c ,'x,yb , c'): return False
        else:
            if G.inv(x)!=0 and G(G.inv(x),c)!=0:
                if not add_conc(y,b, G(G.inv(x),c) ,'y,b, G(G.inv(x),c)'): return False
            #h = set()
            #for (xx,k) in G.decomp(c):
            #    if xx==x:
            #        h.add(k)
            #if not add_hints(y,b , h ): return False

    return True

In [4]:
def fastSetup(G,a,b,c,conc):
    G.setup(a,b,c)
    if (a,b) in conc: del conc[(a,b)]
    return fastCheck(G,a,b,c,conc)
def fastFill(G,conc):
    while len(conc)>0:
        ab = next(iter(conc));
        if not fastSetup(G,ab[0],ab[1],conc[ab],conc):
            return False
    return True
def startCheck(G,conc):
    N = G.N
    for i in range(1,N+1):
        for j in range(1,N+1):
            if G(i,j)!=0:
                if not fastCheck(G,i,j,G(i,j),conc): return False
    if not fastFill(G,conc): return False
    return True

In [5]:
def fastGen(G,ii=1,sets=None):
    """берёт на вход частично заполненную группу и генерирует все пары (список установок, группа /*скопированная*/)"""
    # todo 
    # исключить поиск дыр сделав двунаправленный список из ячеек
    # в каждой ячейке набор кандидатов, при установке значения оно вычеркивается из столбца и строки
    #   с проверкой на единственность оставшихся кандидатов и рекурсивную установку
    # анализ каждой строки на окна ...
    # - это в Group.setup()
    N=G.N
    global DEBUG
    DEBUG = False
    if sets==None: sets = []
    isClean = True # в нем нет дыр
    
    # ищем незаполненную ячейку (дыру)
    i1=0; j1=0
    for i in range(ii,N+1):
        #for j in range(1,N+1):
        #    if G(i,j)==0:
        #        #print(sets,i,j)
        #        isClean = False
        #        i1=i; j1=j
        #        break
        #if not isClean: break
        for j in range(1,i):
            if G(i,j)==0:
                #print(sets,i,j)
                isClean = False
                i1=i; j1=j
                break
        if not isClean: break
        for j in range(1,i+1):
            if G(j,i)==0:
                #print(sets,i,j)
                isClean = False
                i1=j; j1=i
                break
        if not isClean: break
    #print(sets,ii,i1,j1)
    if isClean:
        yield (sets,G)
        #print(sets)
        #print(G)
        #print()
        return 1
    else:
        count = 0
        #print(i1,j1)
        for k in range(1,N+1):
            G1 = copy.deepcopy(G)
            if fastFill(G1,{(i1,j1):k}):
                #count += 
                for q in fastGen(G1,i,[*sets,(i1,j1,k)]): # i->i1
                    yield q
                DEBUG = False
        DEBUG = True
        return count

In [6]:
def checkPerm(p):
    "проверяет, что перестановка состоит из пар, т.к. взаимнообратные элементы - всегда парами"
    for i in range(2,len(p)+2):
        j = p[i-2]
        if p[j-2]!=i:
            return False
    return True

def diaperm(N,k):
    "генерирует перестановку где k последних элементов взаимнообратны попарно, а оставшиеся взаимнообратны сами себе"
    p = [i for i in range(2,N+1)]
    for j in range(k):
        t = p[-2*j-1]
        p[-2*j-1]=p[-2*j-2]
        p[-2*j-2]=t
    return tuple(p)

In [7]:
diaperm(6,1)

(2, 3, 4, 6, 5)

In [8]:
[x for x in fastGen(Group(4,diaperm(4,0)))]

[([(3, 2, 4)],
  1 2 3 4
  2 1 4 3
  3 4 1 2
  4 3 2 1)]

In [9]:
def checkMorphism2(G1,conc,G2):
    ""
    m     = copy.copy(conc)
    conc2 = copy.copy(conc)

    hasNew = True
    while hasNew:
        hasNew = False
        for i in range(1,G1.N+1):
            if m[i-1]==0: continue
            for j in range(1,G1.N+1):
                if m[j-1]==0: continue
                a=m[i-1]
                b=m[j-1]
                if G1(i,j)!=0:
                    if conc[G1(i,j)-1]!=0:
                        c=conc[G1(i,j)-1]
                        if G2(a,b)!=c: 
                            if VERBOSE: print(i,j,G1(i,j),"---",a,b,c)
                            return False
                    else:
                        hasNew = True
                        conc[G1(i,j)-1]=G2(a,b)
        for i in range(G1.N):
            if conc[i]!=0 and conc2[i]==0:
                m[i]=conc[i]
            conc2[i]=conc[i]
    return True

In [10]:
def checkMorphism(G1,conc,G2):
    "проверяет, морфизм ли это, и если он где-то недоопределён - доопределяет"
    hasNew = True
    while hasNew:
        hasNew = False
        for i in range(1,G1.N+1):
            if conc[i-1]==0: continue
            for j in range(1,G1.N+1):
                if conc[j-1]==0: continue
                a=conc[i-1]
                b=conc[j-1]
                if G1(i,j)!=0:
                    if conc[G1(i,j)-1]!=0:
                        c=conc[G1(i,j)-1]
                        if G2(a,b)!=c: 
                            if VERBOSE: print(i,j,G1(i,j),"---",a,b,c)
                            return False
                    else:
                        hasNew = True
                        conc[G1(i,j)-1]=G2(a,b)
    return True

In [11]:
#nest=0
def findMorphism(G1,G2,first,m=None):
    "если first==True - ищет первый морфизм, иначе ищет все морфизмы"
    #global nest
    if m==None:
        m=[0 for i in range(G1.N)]
        m[0]=1
    for i in range(len(m)):
        if m[i]==0:
            break
    if i==len(m)-1 and m[i]!=0:
        if first:
            return m
        else: 
            return {tuple(m)}
    #print('  '*nest,i)
    rr = set()
    for k in range(1,G2.N+1):
        if k in m: continue
        #print('  '*nest,k,rr)
        m[i]=k
        m2 = copy.copy(m)
        if checkMorphism(G1,m2,G2):
            #nest+=1
            r = findMorphism(G1,G2,first,m2)
            #nest-=1
            if r:
                if first: return r
                else: rr|=r
    if first: return False
    else: return rr

In [12]:
def orders(G):
    N = G.N
    o = []
    for i in range(1,N+1):
        cur = i;
        for j in range(1,N+1):
            if cur==1:
                break
            else:
                cur = G(cur,i)
        o.append(j)
    return o

In [13]:
def close(G,els):
    if type(els) is int:
        els = [els]
    eels = list(els)
    i=0
    while i<len(eels):
        a = eels[i]
        assert a<=G.N
        for j in range(i+1):
            b = eels[j]
            c = G(a,b)
            if c!=0 and c not in eels:
                eels.append(c)
        i+=1
    return set(eels)

In [14]:
def closes(G):
    cs = set()
    for i in range(1,G.N+1):
        cs.add(frozenset(close(G,i)))
    return cs

# 2 : 1

In [15]:
N=2
DEBUG = True
VERBOSE = False
Gs = []
for j in range((N-1)//2+1):
    p=diaperm(N,j)
    print('perm:',j,p)
    G = Group(N,p)
    conc = {}
    if not startCheck(G,conc): continue
    print(conc)
    Gs.append([x for x in fastGen(G)])
    #print(G)
Gs


perm: 0 (2,)
{}


[[([],
   1 2
   2 1)]]

In [16]:
G2=Gs[0][0][1]
G2

1 2
2 1

In [17]:
checkMorphism(G2,[2,1],G2)

False

In [18]:
orders(G2)

[1, 2]

# 3 : 1

In [19]:
N=3
DEBUG = True
VERBOSE = False
Gs = []
for j in range((N-1)//2+1):
    p=diaperm(N,j)
    print('perm:',j,p)
    G = Group(N,p)
    conc = {}
    if not startCheck(G,conc): continue
    print(conc)
    Gs.append([x for x in fastGen(G)])
    #print(G)
Gs


perm: 0 (2, 3)
{}
perm: 1 (3, 2)
{}


[[],
 [([(2, 2, 3)], 1 2 3
   2 3 1
   3 1 2)]]

In [20]:
G3=Gs[1][0][1]
G3

1 2 3
2 3 1
3 1 2

In [21]:
checkMorphism(G3,[1,3,2],G3)

True

In [22]:
findMorphism(G3,G3,False)

{(1, 2, 3), (1, 3, 2)}

In [23]:
orders(G3)

[1, 3, 3]

# 4 : 2

In [24]:
N=4
DEBUG = True
VERBOSE = False
Gs = []
for j in range((N-1)//2+1):
    p=diaperm(N,j)
    print('perm:',j,p)
    G = Group(N,p)
    conc = {}
    if not startCheck(G,conc): continue
    print(conc)
    Gs.append([x for x in fastGen(G)])
    #print(G)
Gs


perm: 0 (2, 3, 4)
{}
perm: 1 (2, 4, 3)
{}


[[([(3, 2, 4)],
   1 2 3 4
   2 1 4 3
   3 4 1 2
   4 3 2 1)],
 [([(3, 2, 4)],
   1 2 3 4
   2 1 4 3
   3 4 2 1
   4 3 1 2)]]

In [25]:
G4_0=Gs[0][0][1]
G4_1=Gs[1][0][1]

In [26]:
findMorphism(G2,G4_0,False)

{(1, 2), (1, 3), (1, 4)}

In [27]:
findMorphism(G4_0,G4_0,False)

{(1, 2, 3, 4),
 (1, 2, 4, 3),
 (1, 3, 2, 4),
 (1, 3, 4, 2),
 (1, 4, 2, 3),
 (1, 4, 3, 2)}

In [28]:
findMorphism(G2,G4_1,False)

{(1, 2)}

In [29]:
findMorphism(G4_1,G4_1,False)

{(1, 2, 3, 4), (1, 2, 4, 3)}

In [30]:
orders(G4_0)

[1, 2, 2, 2]

In [31]:
orders(G4_1)

[1, 2, 4, 4]

# 5 : 1

In [32]:
# здесь автоморфизмы ищем вручную
N=5
DEBUG = True
VERBOSE = False
Gs = []
for j in range((N-1)//2+1):
    p=diaperm(N,j)
    print('perm:',j,p)
    G = Group(N,p)
    conc = {}
    if not startCheck(G,conc): continue
    print(conc)
    Gs.append([x for x in fastGen(G)])
    #print(G)
Gs


perm: 0 (2, 3, 4, 5)
{}
perm: 1 (2, 3, 5, 4)
{}
perm: 2 (3, 2, 5, 4)
{}


[[],
 [],
 [([(2, 2, 4), (4, 2, 5)], 1 2 3 4 5
   2 4 1 5 3
   3 1 5 2 4
   4 5 2 3 1
   5 3 4 1 2),
  ([(2, 2, 5), (4, 3, 5)],
   1 2 3 4 5
   2 5 1 3 4
   3 1 4 5 2
   4 3 5 2 1
   5 4 2 1 3)]]

In [33]:
Gs[2][0][1]

1 2 3 4 5
2 4 1 5 3
3 1 5 2 4
4 5 2 3 1
5 3 4 1 2

In [34]:
Gs[2][1][1]

1 2 3 4 5
2 5 1 3 4
3 1 4 5 2
4 3 5 2 1
5 4 2 1 3

In [35]:
findMorphism(Gs[2][0][1],Gs[2][1][1],True)


[1, 2, 3, 5, 4]

In [36]:
G5=Gs[2][0][1]
G5

1 2 3 4 5
2 4 1 5 3
3 1 5 2 4
4 5 2 3 1
5 3 4 1 2

In [37]:
findMorphism(G5,G5,False)


{(1, 2, 3, 4, 5), (1, 3, 2, 5, 4), (1, 4, 5, 3, 2), (1, 5, 4, 2, 3)}

In [38]:
orders(G5)

[1, 5, 5, 5, 5]

# 6 : 2 (1)

In [39]:
# здесь автоморфизмы ищем автоматически
N=6
DEBUG = True
VERBOSE = False
Gs = []
for j in range((N-1)//2+1):
    p=diaperm(N,j)
    print('perm:',j,p)
    G = Group(N,p)
    conc = {}
    if not startCheck(G,conc): continue
    if len(conc)>0 : print(conc)
    Gss = []
    for GG in fastGen(G):
        notNew = False
        for GGG in Gss:
            m = findMorphism(GGG[1],GG[1],True)
            if m:
                notNew = True
                print(GGG[0],m,GG[0])
                break
        if not notNew:
            Gss.append(GG)
            print(GG[0])
            print(GG[1])
    Gs.append(Gss)
Gs


perm: 0 (2, 3, 4, 5, 6)
perm: 1 (2, 3, 4, 6, 5)
[(3, 2, 5), (4, 2, 6), (4, 3, 5)]
1 2 3 4 5 6
2 1 6 5 4 3
3 5 1 6 2 4
4 6 5 1 3 2
5 3 4 2 6 1
6 4 2 3 1 5
[(3, 2, 5), (4, 2, 6), (4, 3, 5)] [1, 2, 3, 4, 6, 5] [(3, 2, 6), (4, 2, 5), (4, 3, 6)]
perm: 2 (2, 4, 3, 6, 5)
[(3, 2, 5), (2, 3, 5), (3, 3, 4)]
1 2 3 4 5 6
2 1 5 6 3 4
3 5 4 1 6 2
4 6 1 3 2 5
5 3 6 2 4 1
6 4 2 5 1 3
[(3, 2, 5), (2, 3, 5), (3, 3, 4)] [1, 2, 5, 6, 3, 4] [(3, 2, 5), (2, 3, 5), (3, 3, 6)]
[(3, 2, 5), (2, 3, 5), (3, 3, 4)] [1, 2, 3, 4, 6, 5] [(3, 2, 6), (2, 3, 6), (3, 3, 4)]
[(3, 2, 5), (2, 3, 5), (3, 3, 4)] [1, 2, 5, 6, 4, 3] [(3, 2, 6), (2, 3, 6), (3, 3, 5)]


[[],
 [([(3, 2, 5), (4, 2, 6), (4, 3, 5)], 1 2 3 4 5 6
   2 1 6 5 4 3
   3 5 1 6 2 4
   4 6 5 1 3 2
   5 3 4 2 6 1
   6 4 2 3 1 5)],
 [([(3, 2, 5), (2, 3, 5), (3, 3, 4)],
   1 2 3 4 5 6
   2 1 5 6 3 4
   3 5 4 1 6 2
   4 6 1 3 2 5
   5 3 6 2 4 1
   6 4 2 5 1 3)]]

In [40]:
G6_1=Gs[1][0][1]
G6_2=Gs[2][0][1]
print(G6_1)
print()
print(G6_2)


1 2 3 4 5 6
2 1 6 5 4 3
3 5 1 6 2 4
4 6 5 1 3 2
5 3 4 2 6 1
6 4 2 3 1 5

1 2 3 4 5 6
2 1 5 6 3 4
3 5 4 1 6 2
4 6 1 3 2 5
5 3 6 2 4 1
6 4 2 5 1 3


In [41]:
findMorphism(G6_1,G6_1,False)


{(1, 2, 3, 4, 5, 6),
 (1, 2, 4, 3, 6, 5),
 (1, 3, 2, 4, 6, 5),
 (1, 3, 4, 2, 5, 6),
 (1, 4, 2, 3, 5, 6),
 (1, 4, 3, 2, 6, 5)}

In [42]:
findMorphism(G6_2,G6_2,False)


{(1, 2, 3, 4, 5, 6), (1, 2, 4, 3, 6, 5)}

In [43]:
findMorphism(G3,G6_1,False)


{(1, 5, 6), (1, 6, 5)}

In [44]:
findMorphism(G3,G6_2,False)


{(1, 3, 4), (1, 4, 3)}

In [45]:
findMorphism(G2,G6_1,False)


{(1, 2), (1, 3), (1, 4)}

In [46]:
findMorphism(G2,G6_2,False)


{(1, 2)}

## G6_1 = Dih3 = S3  !!!

In [48]:
closes(G6_1)

{frozenset({1}),
 frozenset({1, 4}),
 frozenset({1, 2}),
 frozenset({1, 3}),
 frozenset({1, 5, 6})}

## G6_2 = Z6 = Z3 x Z2

In [49]:
closes(G6_2)

{frozenset({1}),
 frozenset({1, 3, 4}),
 frozenset({1, 2}),
 frozenset({1, 2, 3, 4, 5, 6})}

# 7 : 1

In [50]:
N=7
DEBUG = True
VERBOSE = False
Gs = []
for j in range((N-1)//2+1):
    p=diaperm(N,j)
    print('perm:',j,p)
    G = Group(N,p)
    conc = {}
    if not startCheck(G,conc): continue
    if len(conc)>0 : print(conc)
    Gss = []
    for GG in fastGen(G):
        notNew = False
        for GGG in Gss:
            m = findMorphism(GGG[1],GG[1],True)
            if m:
                notNew = True
                print(GGG[0],m,GG[0])
                break
        if not notNew:
            Gss.append(GG)
            print(GG[0])
            print(GG[1])
    Gs.append(Gss)
Gs


perm: 0 (2, 3, 4, 5, 6, 7)
perm: 1 (2, 3, 4, 5, 7, 6)
perm: 2 (2, 3, 5, 4, 7, 6)
perm: 3 (3, 2, 5, 4, 7, 6)
[(2, 2, 4), (4, 2, 6), (4, 4, 7)]
1 2 3 4 5 6 7
2 4 1 6 3 7 5
3 1 5 2 7 4 6
4 6 2 7 1 5 3
5 3 7 1 6 2 4
6 7 4 5 2 3 1
7 5 6 3 4 1 2
[(2, 2, 4), (4, 2, 6), (4, 4, 7)] [1, 2, 3, 4, 5, 7, 6] [(2, 2, 4), (4, 2, 7), (4, 4, 6)]
[(2, 2, 4), (4, 2, 6), (4, 4, 7)] [1, 2, 3, 5, 4, 7, 6] [(2, 2, 5), (4, 3, 6), (4, 4, 7)]
[(2, 2, 4), (4, 2, 6), (4, 4, 7)] [1, 2, 3, 5, 4, 6, 7] [(2, 2, 5), (4, 3, 7), (4, 4, 6)]
[(2, 2, 4), (4, 2, 6), (4, 4, 7)] [1, 2, 3, 6, 7, 4, 5] [(2, 2, 6), (4, 2, 5), (4, 3, 6)]
[(2, 2, 4), (4, 2, 6), (4, 4, 7)] [1, 2, 3, 6, 7, 5, 4] [(2, 2, 6), (4, 2, 7), (4, 3, 5)]
[(2, 2, 4), (4, 2, 6), (4, 4, 7)] [1, 2, 3, 7, 6, 4, 5] [(2, 2, 7), (4, 2, 5), (4, 3, 7)]
[(2, 2, 4), (4, 2, 6), (4, 4, 7)] [1, 2, 3, 7, 6, 5, 4] [(2, 2, 7), (4, 2, 6), (4, 3, 5)]


[[],
 [],
 [],
 [([(2, 2, 4), (4, 2, 6), (4, 4, 7)], 1 2 3 4 5 6 7
   2 4 1 6 3 7 5
   3 1 5 2 7 4 6
   4 6 2 7 1 5 3
   5 3 7 1 6 2 4
   6 7 4 5 2 3 1
   7 5 6 3 4 1 2)]]

In [51]:
G7=Gs[3][0][1]
print(G7)


1 2 3 4 5 6 7
2 4 1 6 3 7 5
3 1 5 2 7 4 6
4 6 2 7 1 5 3
5 3 7 1 6 2 4
6 7 4 5 2 3 1
7 5 6 3 4 1 2


In [52]:
findMorphism(G7,G7,False)


{(1, 2, 3, 4, 5, 6, 7),
 (1, 3, 2, 5, 4, 7, 6),
 (1, 4, 5, 7, 6, 3, 2),
 (1, 5, 4, 6, 7, 2, 3),
 (1, 6, 7, 3, 2, 4, 5),
 (1, 7, 6, 2, 3, 5, 4)}

# 8 : 5 (2)

In [53]:
N=8
DEBUG = True
VERBOSE = False
Gs = []
for j in range((N-1)//2+1):
    p=diaperm(N,j)
    print('perm:',j,p)
    G = Group(N,p)
    conc = {}
    if not startCheck(G,conc): continue
    if len(conc)>0 : print(conc)
    Gss = []
    for GG in fastGen(G):
        notNew = False
        for GGG in Gss:
            m = findMorphism(GGG[1],GG[1],True)
            if m:
                notNew = True
                print(GGG[0],m,GG[0])
                break
        if not notNew:
            Gss.append(GG)
            print(GG[0])
            print(GG[1])
    Gs.append(Gss)
Gs


perm: 0 (2, 3, 4, 5, 6, 7, 8)
[(3, 2, 4), (5, 2, 6), (5, 3, 7), (5, 4, 8)]
1 2 3 4 5 6 7 8
2 1 4 3 6 5 8 7
3 4 1 2 7 8 5 6
4 3 2 1 8 7 6 5
5 6 7 8 1 2 3 4
6 5 8 7 2 1 4 3
7 8 5 6 3 4 1 2
8 7 6 5 4 3 2 1
[(3, 2, 4), (5, 2, 6), (5, 3, 7), (5, 4, 8)] [1, 2, 3, 4, 5, 6, 8, 7] [(3, 2, 4), (5, 2, 6), (5, 3, 8), (5, 4, 7)]
[(3, 2, 4), (5, 2, 6), (5, 3, 7), (5, 4, 8)] [1, 2, 3, 4, 5, 7, 6, 8] [(3, 2, 4), (5, 2, 7), (5, 3, 6), (5, 4, 8)]
[(3, 2, 4), (5, 2, 6), (5, 3, 7), (5, 4, 8)] [1, 2, 3, 4, 5, 7, 8, 6] [(3, 2, 4), (5, 2, 7), (5, 3, 8), (5, 4, 6)]
[(3, 2, 4), (5, 2, 6), (5, 3, 7), (5, 4, 8)] [1, 2, 3, 4, 5, 8, 6, 7] [(3, 2, 4), (5, 2, 8), (5, 3, 6), (5, 4, 7)]
[(3, 2, 4), (5, 2, 6), (5, 3, 7), (5, 4, 8)] [1, 2, 3, 4, 5, 8, 7, 6] [(3, 2, 4), (5, 2, 8), (5, 3, 7), (5, 4, 6)]
[(3, 2, 4), (5, 2, 6), (5, 3, 7), (5, 4, 8)] [1, 2, 3, 5, 4, 6, 7, 8] [(3, 2, 5), (4, 2, 6), (4, 3, 7), (5, 4, 8)]
[(3, 2, 4), (5, 2, 6), (5, 3, 7), (5, 4, 8)] [1, 2, 3, 5, 4, 6, 8, 7] [(3, 2, 5), (4, 2, 6), (4, 3, 8), (5,

[[([(3, 2, 4), (5, 2, 6), (5, 3, 7), (5, 4, 8)],
   1 2 3 4 5 6 7 8
   2 1 4 3 6 5 8 7
   3 4 1 2 7 8 5 6
   4 3 2 1 8 7 6 5
   5 6 7 8 1 2 3 4
   6 5 8 7 2 1 4 3
   7 8 5 6 3 4 1 2
   8 7 6 5 4 3 2 1)],
 [([(3, 2, 4), (5, 2, 6), (5, 3, 7), (5, 4, 8)],
   1 2 3 4 5 6 7 8
   2 1 4 3 6 5 8 7
   3 4 1 2 8 7 6 5
   4 3 2 1 7 8 5 6
   5 6 7 8 1 2 3 4
   6 5 8 7 2 1 4 3
   7 8 5 6 4 3 2 1
   8 7 6 5 3 4 1 2)],
 [([(3, 2, 4), (5, 2, 6), (5, 3, 7), (5, 4, 8)],
   1 2 3 4 5 6 7 8
   2 1 4 3 6 5 8 7
   3 4 1 2 7 8 5 6
   4 3 2 1 8 7 6 5
   5 6 7 8 2 1 4 3
   6 5 8 7 1 2 3 4
   7 8 5 6 4 3 2 1
   8 7 6 5 3 4 1 2)],
 [([(3, 2, 4), (5, 2, 6), (5, 3, 7), (5, 4, 8)],
   1 2 3 4 5 6 7 8
   2 1 4 3 6 5 8 7
   3 4 2 1 8 7 5 6
   4 3 1 2 7 8 6 5
   5 6 7 8 2 1 4 3
   6 5 8 7 1 2 3 4
   7 8 6 5 3 4 2 1
   8 7 5 6 4 3 1 2),
  ([(3, 2, 4), (5, 2, 7), (5, 3, 6)],
   1 2 3 4 5 6 7 8
   2 1 4 3 7 8 5 6
   3 4 2 1 6 7 8 5
   4 3 1 2 8 5 6 7
   5 7 6 8 4 1 3 2
   6 8 7 5 1 3 2 4
   7 5 8 6 3 2 4 1
   8 6 5 7 2 4

In [54]:
G8_5=Gs[0][0][1]
G8_3=Gs[1][0][1]
G8_2=Gs[2][0][1]
G8_4=Gs[3][0][1]
G8_1=Gs[3][1][1]


## G8_1 = Z8

In [55]:
G8_1

1 2 3 4 5 6 7 8
2 1 4 3 7 8 5 6
3 4 2 1 6 7 8 5
4 3 1 2 8 5 6 7
5 7 6 8 4 1 3 2
6 8 7 5 1 3 2 4
7 5 8 6 3 2 4 1
8 6 5 7 2 4 1 3

In [56]:
closes(G8_1)

{frozenset({1}),
 frozenset({1, 2}),
 frozenset({1, 2, 3, 4}),
 frozenset({1, 2, 3, 4, 5, 6, 7, 8})}

## G8_4 = Dic2  !!!

In [57]:
G8_2

1 2 3 4 5 6 7 8
2 1 4 3 6 5 8 7
3 4 1 2 7 8 5 6
4 3 2 1 8 7 6 5
5 6 7 8 2 1 4 3
6 5 8 7 1 2 3 4
7 8 5 6 4 3 2 1
8 7 6 5 3 4 1 2

In [58]:
closes(G8_2)

{frozenset({1}),
 frozenset({1, 4}),
 frozenset({1, 2}),
 frozenset({1, 2, 5, 6}),
 frozenset({1, 2, 7, 8}),
 frozenset({1, 3})}

## G8_2 = Z4 x Z2

In [59]:
G8_2

1 2 3 4 5 6 7 8
2 1 4 3 6 5 8 7
3 4 1 2 7 8 5 6
4 3 2 1 8 7 6 5
5 6 7 8 2 1 4 3
6 5 8 7 1 2 3 4
7 8 5 6 4 3 2 1
8 7 6 5 3 4 1 2

In [60]:
closes(G8_2)

{frozenset({1}),
 frozenset({1, 4}),
 frozenset({1, 2}),
 frozenset({1, 2, 5, 6}),
 frozenset({1, 2, 7, 8}),
 frozenset({1, 3})}

## G8_3 = Dih4   !!!

In [61]:
G8_3

1 2 3 4 5 6 7 8
2 1 4 3 6 5 8 7
3 4 1 2 8 7 6 5
4 3 2 1 7 8 5 6
5 6 7 8 1 2 3 4
6 5 8 7 2 1 4 3
7 8 5 6 4 3 2 1
8 7 6 5 3 4 1 2

In [62]:
closes(G8_3)

{frozenset({1}),
 frozenset({1, 4}),
 frozenset({1, 2}),
 frozenset({1, 5}),
 frozenset({1, 6}),
 frozenset({1, 2, 7, 8}),
 frozenset({1, 3})}

## G8_5 = Z2 x Z2 x Z2

In [63]:
G8_5

1 2 3 4 5 6 7 8
2 1 4 3 6 5 8 7
3 4 1 2 7 8 5 6
4 3 2 1 8 7 6 5
5 6 7 8 1 2 3 4
6 5 8 7 2 1 4 3
7 8 5 6 3 4 1 2
8 7 6 5 4 3 2 1

In [64]:
closes(G8_5)

{frozenset({1}),
 frozenset({1, 4}),
 frozenset({1, 7}),
 frozenset({1, 2}),
 frozenset({1, 5}),
 frozenset({1, 8}),
 frozenset({1, 6}),
 frozenset({1, 3})}

## прочее

In [71]:
findMorphism(G8_5,G8_5,False)

{(1, 2, 3, 4, 5, 6, 7, 8),
 (1, 2, 3, 4, 6, 5, 8, 7),
 (1, 2, 3, 4, 7, 8, 5, 6),
 (1, 2, 3, 4, 8, 7, 6, 5),
 (1, 2, 4, 3, 5, 6, 8, 7),
 (1, 2, 4, 3, 6, 5, 7, 8),
 (1, 2, 4, 3, 7, 8, 6, 5),
 (1, 2, 4, 3, 8, 7, 5, 6),
 (1, 2, 5, 6, 3, 4, 7, 8),
 (1, 2, 5, 6, 4, 3, 8, 7),
 (1, 2, 5, 6, 7, 8, 3, 4),
 (1, 2, 5, 6, 8, 7, 4, 3),
 (1, 2, 6, 5, 3, 4, 8, 7),
 (1, 2, 6, 5, 4, 3, 7, 8),
 (1, 2, 6, 5, 7, 8, 4, 3),
 (1, 2, 6, 5, 8, 7, 3, 4),
 (1, 2, 7, 8, 3, 4, 5, 6),
 (1, 2, 7, 8, 4, 3, 6, 5),
 (1, 2, 7, 8, 5, 6, 3, 4),
 (1, 2, 7, 8, 6, 5, 4, 3),
 (1, 2, 8, 7, 3, 4, 6, 5),
 (1, 2, 8, 7, 4, 3, 5, 6),
 (1, 2, 8, 7, 5, 6, 4, 3),
 (1, 2, 8, 7, 6, 5, 3, 4),
 (1, 3, 2, 4, 5, 7, 6, 8),
 (1, 3, 2, 4, 6, 8, 5, 7),
 (1, 3, 2, 4, 7, 5, 8, 6),
 (1, 3, 2, 4, 8, 6, 7, 5),
 (1, 3, 4, 2, 5, 7, 8, 6),
 (1, 3, 4, 2, 6, 8, 7, 5),
 (1, 3, 4, 2, 7, 5, 6, 8),
 (1, 3, 4, 2, 8, 6, 5, 7),
 (1, 3, 5, 7, 2, 4, 6, 8),
 (1, 3, 5, 7, 4, 2, 8, 6),
 (1, 3, 5, 7, 6, 8, 2, 4),
 (1, 3, 5, 7, 8, 6, 4, 2),
 (1, 3, 6, 8, 2, 4, 5, 7),
 

In [72]:
findMorphism(G4_0,G8_5,False)

{(1, 2, 3, 4),
 (1, 2, 4, 3),
 (1, 2, 5, 6),
 (1, 2, 6, 5),
 (1, 2, 7, 8),
 (1, 2, 8, 7),
 (1, 3, 2, 4),
 (1, 3, 4, 2),
 (1, 3, 5, 7),
 (1, 3, 6, 8),
 (1, 3, 7, 5),
 (1, 3, 8, 6),
 (1, 4, 2, 3),
 (1, 4, 3, 2),
 (1, 4, 5, 8),
 (1, 4, 6, 7),
 (1, 4, 7, 6),
 (1, 4, 8, 5),
 (1, 5, 2, 6),
 (1, 5, 3, 7),
 (1, 5, 4, 8),
 (1, 5, 6, 2),
 (1, 5, 7, 3),
 (1, 5, 8, 4),
 (1, 6, 2, 5),
 (1, 6, 3, 8),
 (1, 6, 4, 7),
 (1, 6, 5, 2),
 (1, 6, 7, 4),
 (1, 6, 8, 3),
 (1, 7, 2, 8),
 (1, 7, 3, 5),
 (1, 7, 4, 6),
 (1, 7, 5, 3),
 (1, 7, 6, 4),
 (1, 7, 8, 2),
 (1, 8, 2, 7),
 (1, 8, 3, 6),
 (1, 8, 4, 5),
 (1, 8, 5, 4),
 (1, 8, 6, 3),
 (1, 8, 7, 2)}

In [73]:
G8_3

1 2 3 4 5 6 7 8
2 1 4 3 6 5 8 7
3 4 1 2 8 7 6 5
4 3 2 1 7 8 5 6
5 6 7 8 1 2 3 4
6 5 8 7 2 1 4 3
7 8 5 6 4 3 2 1
8 7 6 5 3 4 1 2

In [74]:
findMorphism(G8_3,G8_3,False)

{(1, 2, 3, 4, 5, 6, 7, 8),
 (1, 2, 3, 4, 6, 5, 8, 7),
 (1, 2, 4, 3, 5, 6, 8, 7),
 (1, 2, 4, 3, 6, 5, 7, 8),
 (1, 2, 5, 6, 3, 4, 8, 7),
 (1, 2, 5, 6, 4, 3, 7, 8),
 (1, 2, 6, 5, 3, 4, 7, 8),
 (1, 2, 6, 5, 4, 3, 8, 7)}

In [75]:
findMorphism(G4_0,G8_3,False)

{(1, 2, 3, 4),
 (1, 2, 4, 3),
 (1, 2, 5, 6),
 (1, 2, 6, 5),
 (1, 3, 2, 4),
 (1, 3, 4, 2),
 (1, 4, 2, 3),
 (1, 4, 3, 2),
 (1, 5, 2, 6),
 (1, 5, 6, 2),
 (1, 6, 2, 5),
 (1, 6, 5, 2)}

In [76]:
findMorphism(G4_1,G8_3,False)

{(1, 2, 7, 8), (1, 2, 8, 7)}

In [77]:
G8_2

1 2 3 4 5 6 7 8
2 1 4 3 6 5 8 7
3 4 1 2 7 8 5 6
4 3 2 1 8 7 6 5
5 6 7 8 2 1 4 3
6 5 8 7 1 2 3 4
7 8 5 6 4 3 2 1
8 7 6 5 3 4 1 2

In [78]:
findMorphism(G8_2,G8_2,False)

{(1, 2, 3, 4, 5, 6, 7, 8),
 (1, 2, 3, 4, 6, 5, 8, 7),
 (1, 2, 3, 4, 7, 8, 5, 6),
 (1, 2, 3, 4, 8, 7, 6, 5),
 (1, 2, 4, 3, 5, 6, 8, 7),
 (1, 2, 4, 3, 6, 5, 7, 8),
 (1, 2, 4, 3, 7, 8, 6, 5),
 (1, 2, 4, 3, 8, 7, 5, 6)}

In [79]:
findMorphism(G4_0,G8_2,False)

{(1, 2, 3, 4),
 (1, 2, 4, 3),
 (1, 3, 2, 4),
 (1, 3, 4, 2),
 (1, 4, 2, 3),
 (1, 4, 3, 2)}

In [80]:
findMorphism(G4_1,G8_2,False)

{(1, 2, 5, 6), (1, 2, 6, 5), (1, 2, 7, 8), (1, 2, 8, 7)}

In [82]:
findMorphism(G8_4,G8_4,False)

{(1, 2, 3, 4, 5, 6, 7, 8),
 (1, 2, 3, 4, 6, 5, 8, 7),
 (1, 2, 3, 4, 7, 8, 6, 5),
 (1, 2, 3, 4, 8, 7, 5, 6),
 (1, 2, 4, 3, 5, 6, 8, 7),
 (1, 2, 4, 3, 6, 5, 7, 8),
 (1, 2, 4, 3, 7, 8, 5, 6),
 (1, 2, 4, 3, 8, 7, 6, 5),
 (1, 2, 5, 6, 3, 4, 8, 7),
 (1, 2, 5, 6, 4, 3, 7, 8),
 (1, 2, 5, 6, 7, 8, 3, 4),
 (1, 2, 5, 6, 8, 7, 4, 3),
 (1, 2, 6, 5, 3, 4, 7, 8),
 (1, 2, 6, 5, 4, 3, 8, 7),
 (1, 2, 6, 5, 7, 8, 4, 3),
 (1, 2, 6, 5, 8, 7, 3, 4),
 (1, 2, 7, 8, 3, 4, 5, 6),
 (1, 2, 7, 8, 4, 3, 6, 5),
 (1, 2, 7, 8, 5, 6, 4, 3),
 (1, 2, 7, 8, 6, 5, 3, 4),
 (1, 2, 8, 7, 3, 4, 6, 5),
 (1, 2, 8, 7, 4, 3, 5, 6),
 (1, 2, 8, 7, 5, 6, 3, 4),
 (1, 2, 8, 7, 6, 5, 4, 3)}

In [83]:
G8_4

1 2 3 4 5 6 7 8
2 1 4 3 6 5 8 7
3 4 2 1 8 7 5 6
4 3 1 2 7 8 6 5
5 6 7 8 2 1 4 3
6 5 8 7 1 2 3 4
7 8 6 5 3 4 2 1
8 7 5 6 4 3 1 2

In [84]:
findMorphism(G8_4,G8_4,False)

{(1, 2, 3, 4, 5, 6, 7, 8),
 (1, 2, 3, 4, 6, 5, 8, 7),
 (1, 2, 3, 4, 7, 8, 6, 5),
 (1, 2, 3, 4, 8, 7, 5, 6),
 (1, 2, 4, 3, 5, 6, 8, 7),
 (1, 2, 4, 3, 6, 5, 7, 8),
 (1, 2, 4, 3, 7, 8, 5, 6),
 (1, 2, 4, 3, 8, 7, 6, 5),
 (1, 2, 5, 6, 3, 4, 8, 7),
 (1, 2, 5, 6, 4, 3, 7, 8),
 (1, 2, 5, 6, 7, 8, 3, 4),
 (1, 2, 5, 6, 8, 7, 4, 3),
 (1, 2, 6, 5, 3, 4, 7, 8),
 (1, 2, 6, 5, 4, 3, 8, 7),
 (1, 2, 6, 5, 7, 8, 4, 3),
 (1, 2, 6, 5, 8, 7, 3, 4),
 (1, 2, 7, 8, 3, 4, 5, 6),
 (1, 2, 7, 8, 4, 3, 6, 5),
 (1, 2, 7, 8, 5, 6, 4, 3),
 (1, 2, 7, 8, 6, 5, 3, 4),
 (1, 2, 8, 7, 3, 4, 6, 5),
 (1, 2, 8, 7, 4, 3, 5, 6),
 (1, 2, 8, 7, 5, 6, 3, 4),
 (1, 2, 8, 7, 6, 5, 4, 3)}

In [85]:
findMorphism(G4_0,G8_4,False)

set()

In [86]:
findMorphism(G4_1,G8_4,False)

{(1, 2, 3, 4),
 (1, 2, 4, 3),
 (1, 2, 5, 6),
 (1, 2, 6, 5),
 (1, 2, 7, 8),
 (1, 2, 8, 7)}

In [87]:
G8_1

1 2 3 4 5 6 7 8
2 1 4 3 7 8 5 6
3 4 2 1 6 7 8 5
4 3 1 2 8 5 6 7
5 7 6 8 4 1 3 2
6 8 7 5 1 3 2 4
7 5 8 6 3 2 4 1
8 6 5 7 2 4 1 3

In [89]:
findMorphism(G8_1,G8_1,False)

{(1, 2, 3, 4, 5, 6, 7, 8),
 (1, 2, 3, 4, 7, 8, 5, 6),
 (1, 2, 4, 3, 6, 5, 8, 7),
 (1, 2, 4, 3, 8, 7, 6, 5)}

In [90]:
findMorphism(G4_0,G8_1,False)

set()

In [91]:
findMorphism(G4_1,G8_1,False)

{(1, 2, 3, 4), (1, 2, 4, 3)}

# 9 : 2

In [92]:
N=9
DEBUG = True
VERBOSE = False
Gs = []
for j in range((N-1)//2+1):
    p=diaperm(N,j)
    print('perm:',j,p)
    G = Group(N,p)
    conc = {}
    if not startCheck(G,conc): continue
    if len(conc)>0 : print(conc)
    Gss = []
    for GG in fastGen(G):
        notNew = False
        for GGG in Gss:
            m = findMorphism(GGG[1],GG[1],True)
            if m:
                notNew = True
                print(GGG[0],m,GG[0])
                break
        if not notNew:
            Gss.append(GG)
            print(GG[0])
            print(GG[1])
    Gs.append(Gss)
Gs


perm: 0 (2, 3, 4, 5, 6, 7, 8, 9)
perm: 1 (2, 3, 4, 5, 6, 7, 9, 8)
perm: 2 (2, 3, 4, 5, 7, 6, 9, 8)
perm: 3 (2, 3, 5, 4, 7, 6, 9, 8)
perm: 4 (3, 2, 5, 4, 7, 6, 9, 8)
[(2, 2, 3), (4, 2, 6), (4, 3, 8), (2, 4, 6), (4, 4, 5)]
1 2 3 4 5 6 7 8 9
2 3 1 6 9 8 5 4 7
3 1 2 8 7 4 9 6 5
4 6 8 5 1 9 3 7 2
5 9 7 1 4 2 8 3 6
6 8 4 9 2 7 1 5 3
7 5 9 3 8 1 6 2 4
8 4 6 7 3 5 2 9 1
9 7 5 2 6 3 4 1 8
[(2, 2, 3), (4, 2, 6), (4, 3, 8), (2, 4, 6), (4, 4, 7)]
1 2 3 4 5 6 7 8 9
2 3 1 6 9 8 5 4 7
3 1 2 8 7 4 9 6 5
4 6 8 7 1 5 3 9 2
5 9 7 1 6 2 4 3 8
6 8 4 5 2 9 1 7 3
7 5 9 3 4 1 8 2 6
8 4 6 9 3 7 2 5 1
9 7 5 2 8 3 6 1 4
[(2, 2, 3), (4, 2, 6), (4, 3, 8), (2, 4, 6), (4, 4, 7)] [1, 2, 3, 5, 4, 9, 8, 7, 6] [(2, 2, 3), (4, 2, 6), (4, 3, 8), (2, 4, 6), (4, 4, 9)]
[(2, 2, 3), (4, 2, 6), (4, 3, 8), (2, 4, 6), (4, 4, 5)] [1, 2, 3, 4, 5, 6, 7, 9, 8] [(2, 2, 3), (4, 2, 6), (4, 3, 9), (2, 4, 6), (4, 4, 5)]
[(2, 2, 3), (4, 2, 6), (4, 3, 8), (2, 4, 6), (4, 4, 7)] [1, 2, 3, 4, 5, 6, 7, 9, 8] [(2, 2, 3), (4, 2, 6), (4, 3, 9), (

[[],
 [],
 [],
 [],
 [([(2, 2, 3), (4, 2, 6), (4, 3, 8), (2, 4, 6), (4, 4, 5)],
   1 2 3 4 5 6 7 8 9
   2 3 1 6 9 8 5 4 7
   3 1 2 8 7 4 9 6 5
   4 6 8 5 1 9 3 7 2
   5 9 7 1 4 2 8 3 6
   6 8 4 9 2 7 1 5 3
   7 5 9 3 8 1 6 2 4
   8 4 6 7 3 5 2 9 1
   9 7 5 2 6 3 4 1 8),
  ([(2, 2, 3), (4, 2, 6), (4, 3, 8), (2, 4, 6), (4, 4, 7)],
   1 2 3 4 5 6 7 8 9
   2 3 1 6 9 8 5 4 7
   3 1 2 8 7 4 9 6 5
   4 6 8 7 1 5 3 9 2
   5 9 7 1 6 2 4 3 8
   6 8 4 5 2 9 1 7 3
   7 5 9 3 4 1 8 2 6
   8 4 6 9 3 7 2 5 1
   9 7 5 2 8 3 6 1 4)]]

In [93]:
G9_1 = Gs[4][0][1]
G9_2 = Gs[4][1][1]


## G9_1 = Z9

In [94]:
G9_1

1 2 3 4 5 6 7 8 9
2 3 1 6 9 8 5 4 7
3 1 2 8 7 4 9 6 5
4 6 8 5 1 9 3 7 2
5 9 7 1 4 2 8 3 6
6 8 4 9 2 7 1 5 3
7 5 9 3 8 1 6 2 4
8 4 6 7 3 5 2 9 1
9 7 5 2 6 3 4 1 8

In [95]:
closes(G9_1)

{frozenset({1}),
 frozenset({1, 6, 7}),
 frozenset({1, 8, 9}),
 frozenset({1, 4, 5}),
 frozenset({1, 2, 3})}

## G9_2 = Z3 x Z3

In [96]:
G9_2

1 2 3 4 5 6 7 8 9
2 3 1 6 9 8 5 4 7
3 1 2 8 7 4 9 6 5
4 6 8 7 1 5 3 9 2
5 9 7 1 6 2 4 3 8
6 8 4 5 2 9 1 7 3
7 5 9 3 4 1 8 2 6
8 4 6 9 3 7 2 5 1
9 7 5 2 8 3 6 1 4

In [97]:
closes(G9_2)

{frozenset({1}), frozenset({1, 2, 3}), frozenset({1, 2, 3, 4, 5, 6, 7, 8, 9})}

## прочее

In [98]:
findMorphism(G9_1,G9_2,False)

set()

In [99]:
findMorphism(G9_1,G9_1,False)

{(1, 2, 3, 4, 5, 6, 7, 8, 9),
 (1, 2, 3, 5, 4, 9, 8, 7, 6),
 (1, 2, 3, 6, 7, 8, 9, 4, 5),
 (1, 2, 3, 7, 6, 5, 4, 9, 8),
 (1, 2, 3, 8, 9, 4, 5, 6, 7),
 (1, 2, 3, 9, 8, 7, 6, 5, 4),
 (1, 3, 2, 4, 5, 8, 9, 6, 7),
 (1, 3, 2, 5, 4, 7, 6, 9, 8),
 (1, 3, 2, 6, 7, 4, 5, 8, 9),
 (1, 3, 2, 7, 6, 9, 8, 5, 4),
 (1, 3, 2, 8, 9, 6, 7, 4, 5),
 (1, 3, 2, 9, 8, 5, 4, 7, 6),
 (1, 4, 5, 2, 3, 6, 7, 9, 8),
 (1, 4, 5, 3, 2, 8, 9, 7, 6),
 (1, 4, 5, 6, 7, 9, 8, 2, 3),
 (1, 4, 5, 7, 6, 3, 2, 8, 9),
 (1, 4, 5, 8, 9, 7, 6, 3, 2),
 (1, 4, 5, 9, 8, 2, 3, 6, 7),
 (1, 5, 4, 2, 3, 9, 8, 6, 7),
 (1, 5, 4, 3, 2, 7, 6, 8, 9),
 (1, 5, 4, 6, 7, 2, 3, 9, 8),
 (1, 5, 4, 7, 6, 8, 9, 3, 2),
 (1, 5, 4, 8, 9, 3, 2, 7, 6),
 (1, 5, 4, 9, 8, 6, 7, 2, 3),
 (1, 6, 7, 2, 3, 8, 9, 5, 4),
 (1, 6, 7, 3, 2, 4, 5, 9, 8),
 (1, 6, 7, 4, 5, 9, 8, 3, 2),
 (1, 6, 7, 5, 4, 2, 3, 8, 9),
 (1, 6, 7, 8, 9, 5, 4, 2, 3),
 (1, 6, 7, 9, 8, 3, 2, 4, 5),
 (1, 7, 6, 2, 3, 5, 4, 8, 9),
 (1, 7, 6, 3, 2, 9, 8, 4, 5),
 (1, 7, 6, 4, 5, 3, 2, 9, 8),
 (1, 7, 6,

In [100]:
findMorphism(G3,G9_1,False)

{(1, 2, 3),
 (1, 3, 2),
 (1, 4, 5),
 (1, 5, 4),
 (1, 6, 7),
 (1, 7, 6),
 (1, 8, 9),
 (1, 9, 8)}

In [101]:
findMorphism(G9_2,G9_2,False)

{(1, 2, 3, 4, 5, 6, 7, 8, 9),
 (1, 2, 3, 6, 7, 8, 9, 4, 5),
 (1, 2, 3, 8, 9, 4, 5, 6, 7),
 (1, 3, 2, 5, 4, 7, 6, 9, 8),
 (1, 3, 2, 7, 6, 9, 8, 5, 4),
 (1, 3, 2, 9, 8, 5, 4, 7, 6)}

In [102]:
findMorphism(G3,G9_2,False)

{(1, 2, 3), (1, 3, 2)}

# 10 : 2 (1)

## G10_2 = Z10

In [103]:
G10_2 = Group("""
   01 02 03 04 05 06 07 08 09 10
   02 01 05 06 03 04 09 10 07 08
   03 05 07 01 09 02 08 04 10 06
   04 06 01 08 02 10 03 07 05 09
   05 03 09 02 07 01 10 06 08 04
   06 04 02 10 01 08 05 09 03 07
   07 09 08 03 10 05 04 01 06 02
   08 10 04 07 06 09 01 03 02 05
   09 07 10 05 08 03 06 02 04 01
   10 08 06 09 04 07 02 05 01 03""")

In [104]:
closes(G10_2)

{frozenset({1}),
 frozenset({1, 3, 4, 7, 8}),
 frozenset({1, 2}),
 frozenset({1, 2, 3, 4, 5, 6, 7, 8, 9, 10})}

In [105]:
closes(G)

{frozenset({2}),
 frozenset({9}),
 frozenset({8}),
 frozenset({7}),
 frozenset({3}),
 frozenset({1}),
 frozenset({6}),
 frozenset({5}),
 frozenset({4})}

## G10_1 = Dih5  !!!

In [106]:
G10_1 = Group("""   01 02 03 04 05 06 07 08 09 10
   02 01 08 07 10 09 04 03 06 05
   03 07 01 10 09 08 02 06 05 04
   04 08 09 01 07 10 05 02 03 06
   05 09 10 08 01 07 06 04 02 03
   06 10 07 09 08 01 03 05 04 02
   07 03 06 02 04 05 10 01 08 09
   08 04 02 05 06 03 01 09 10 07
   09 05 04 06 03 02 08 10 07 01
   10 06 05 03 02 04 09 07 01 08""")

In [107]:
closes(G10_1)

{frozenset({1}),
 frozenset({1, 4}),
 frozenset({1, 2}),
 frozenset({1, 5}),
 frozenset({1, 7, 8, 9, 10}),
 frozenset({1, 6}),
 frozenset({1, 3})}

## прочее

In [108]:
N=10
DEBUG = True
VERBOSE = False
Gs = []
for j in range((N-1)//2+1):
    p=diaperm(N,j)
    print('perm:',j,p)
    G = Group(N,p)
    conc = {}
    if not startCheck(G,conc): continue
    if len(conc)>0 : print(conc)
    Gss = []
    for GG in fastGen(G):
        notNew = False
        for GGG in Gss:
            m = findMorphism(GGG[1],GG[1],True)
            if m:
                notNew = True
                print(GGG[0],m,GG[0])
                break
        if not notNew:
            Gss.append(GG)
            print(GG[0])
            print(GG[1])
    Gs.append(Gss)
Gs
# executed in 31.2s, finished 10:51:56 2023-02-02

perm: 0 (2, 3, 4, 5, 6, 7, 8, 9, 10)
perm: 1 (2, 3, 4, 5, 6, 7, 8, 10, 9)
perm: 2 (2, 3, 4, 5, 6, 8, 7, 10, 9)
[(3, 2, 7), (4, 2, 8), (4, 3, 9), (5, 2, 9), (5, 3, 10), (6, 2, 10)]
01 02 03 04 05 06 07 08 09 10
02 01 08 07 10 09 04 03 06 05
03 07 01 10 09 08 02 06 05 04
04 08 09 01 07 10 05 02 03 06
05 09 10 08 01 07 06 04 02 03
06 10 07 09 08 01 03 05 04 02
07 03 06 02 04 05 10 01 08 09
08 04 02 05 06 03 01 09 10 07
09 05 04 06 03 02 08 10 07 01
10 06 05 03 02 04 09 07 01 08
[(3, 2, 7), (4, 2, 8), (4, 3, 9), (5, 2, 9), (5, 3, 10), (6, 2, 10)] [1, 2, 3, 4, 6, 5, 7, 8, 9, 10] [(3, 2, 7), (4, 2, 8), (4, 3, 9), (5, 2, 10), (5, 4, 9), (6, 2, 9)]
[(3, 2, 7), (4, 2, 8), (4, 3, 9), (5, 2, 9), (5, 3, 10), (6, 2, 10)] [1, 2, 3, 4, 6, 5, 7, 8, 10, 9] [(3, 2, 7), (4, 2, 8), (4, 3, 10), (5, 2, 9), (5, 4, 10), (6, 2, 10)]
[(3, 2, 7), (4, 2, 8), (4, 3, 9), (5, 2, 9), (5, 3, 10), (6, 2, 10)] [1, 2, 3, 4, 5, 6, 7, 8, 10, 9] [(3, 2, 7), (4, 2, 8), (4, 3, 10), (5, 2, 10), (5, 3, 9), (6, 2, 9)]
[(3, 2, 7)

[[],
 [],
 [([(3, 2, 7), (4, 2, 8), (4, 3, 9), (5, 2, 9), (5, 3, 10), (6, 2, 10)],
   01 02 03 04 05 06 07 08 09 10
   02 01 08 07 10 09 04 03 06 05
   03 07 01 10 09 08 02 06 05 04
   04 08 09 01 07 10 05 02 03 06
   05 09 10 08 01 07 06 04 02 03
   06 10 07 09 08 01 03 05 04 02
   07 03 06 02 04 05 10 01 08 09
   08 04 02 05 06 03 01 09 10 07
   09 05 04 06 03 02 08 10 07 01
   10 06 05 03 02 04 09 07 01 08)],
 [],
 [([(3, 2, 5), (2, 3, 5), (3, 3, 7), (5, 3, 9), (7, 3, 8)],
   01 02 03 04 05 06 07 08 09 10
   02 01 05 06 03 04 09 10 07 08
   03 05 07 01 09 02 08 04 10 06
   04 06 01 08 02 10 03 07 05 09
   05 03 09 02 07 01 10 06 08 04
   06 04 02 10 01 08 05 09 03 07
   07 09 08 03 10 05 04 01 06 02
   08 10 04 07 06 09 01 03 02 05
   09 07 10 05 08 03 06 02 04 01
   10 08 06 09 04 07 02 05 01 03)]]

In [109]:
G10_2 = Gs[2][0][1]
G10_4 = Gs[4][0][1]


In [110]:
findMorphism(G10_2,G10_2,False)

{(1, 2, 3, 4, 5, 6, 7, 8, 9, 10),
 (1, 2, 4, 3, 6, 5, 8, 7, 10, 9),
 (1, 2, 5, 6, 4, 3, 9, 10, 8, 7),
 (1, 2, 6, 5, 3, 4, 10, 9, 7, 8),
 (1, 3, 2, 6, 5, 4, 8, 7, 10, 9),
 (1, 3, 4, 5, 2, 6, 9, 10, 8, 7),
 (1, 3, 5, 4, 6, 2, 10, 9, 7, 8),
 (1, 3, 6, 2, 4, 5, 7, 8, 9, 10),
 (1, 4, 2, 5, 6, 3, 7, 8, 9, 10),
 (1, 4, 3, 6, 2, 5, 10, 9, 7, 8),
 (1, 4, 5, 2, 3, 6, 8, 7, 10, 9),
 (1, 4, 6, 3, 5, 2, 9, 10, 8, 7),
 (1, 5, 2, 3, 4, 6, 10, 9, 7, 8),
 (1, 5, 3, 2, 6, 4, 9, 10, 8, 7),
 (1, 5, 4, 6, 3, 2, 7, 8, 9, 10),
 (1, 5, 6, 4, 2, 3, 8, 7, 10, 9),
 (1, 6, 2, 4, 3, 5, 9, 10, 8, 7),
 (1, 6, 3, 5, 4, 2, 8, 7, 10, 9),
 (1, 6, 4, 2, 5, 3, 10, 9, 7, 8),
 (1, 6, 5, 3, 2, 4, 7, 8, 9, 10)}

In [111]:
findMorphism(G5,G10_2,False)

{(1, 7, 8, 10, 9), (1, 8, 7, 9, 10), (1, 9, 10, 7, 8), (1, 10, 9, 8, 7)}

In [112]:
findMorphism(G2,G10_2,False)

{(1, 2), (1, 3), (1, 4), (1, 5), (1, 6)}

In [113]:
findMorphism(G10_4,G10_4,False)

{(1, 2, 3, 4, 5, 6, 7, 8, 9, 10),
 (1, 2, 4, 3, 6, 5, 8, 7, 10, 9),
 (1, 2, 7, 8, 9, 10, 4, 3, 6, 5),
 (1, 2, 8, 7, 10, 9, 3, 4, 5, 6)}

In [114]:
findMorphism(G5,G10_4,False)

{(1, 3, 4, 7, 8), (1, 4, 3, 8, 7), (1, 7, 8, 4, 3), (1, 8, 7, 3, 4)}

In [115]:
findMorphism(G2,G10_4,False)

{(1, 2)}

# 11 : 1

In [116]:
N=11
DEBUG = True
VERBOSE = False
Gs = []
for j in range((N-1)//2+1):
    p=diaperm(N,j)
    print('perm:',j,p)
    G = Group(N,p)
    conc = {}
    if not startCheck(G,conc): continue
    if len(conc)>0 : print(conc)
    Gss = []
    for GG in fastGen(G):
        notNew = False
        for GGG in Gss:
            m = findMorphism(GGG[1],GG[1],True)
            if m:
                notNew = True
                print(GGG[0],m,GG[0])
                break
        if not notNew:
            Gss.append(GG)
            print(GG[0])
            print(GG[1])
    Gs.append(Gss)
Gs
# executed in 3m 30s, finished 10:59:50 2023-02-02

perm: 0 (2, 3, 4, 5, 6, 7, 8, 9, 10, 11)
perm: 1 (2, 3, 4, 5, 6, 7, 8, 9, 11, 10)
perm: 2 (2, 3, 4, 5, 6, 7, 9, 8, 11, 10)
perm: 3 (2, 3, 4, 5, 7, 6, 9, 8, 11, 10)
perm: 4 (2, 3, 5, 4, 7, 6, 9, 8, 11, 10)
perm: 5 (3, 2, 5, 4, 7, 6, 9, 8, 11, 10)
[(2, 2, 4), (4, 2, 6), (4, 4, 8), (6, 4, 10), (6, 6, 11)]
01 02 03 04 05 06 07 08 09 10 11
02 04 01 06 03 08 05 10 07 11 09
03 01 05 02 07 04 09 06 11 08 10
04 06 02 08 01 10 03 11 05 09 07
05 03 07 01 09 02 11 04 10 06 08
06 08 04 10 02 11 01 09 03 07 05
07 05 09 03 11 01 10 02 08 04 06
08 10 06 11 04 09 02 07 01 05 03
09 07 11 05 10 03 08 01 06 02 04
10 11 08 09 06 07 04 05 02 03 01
11 09 10 07 08 05 06 03 04 01 02
[(2, 2, 4), (4, 2, 6), (4, 4, 8), (6, 4, 10), (6, 6, 11)] [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 10] [(2, 2, 4), (4, 2, 6), (4, 4, 8), (6, 4, 11), (6, 6, 10)]
[(2, 2, 4), (4, 2, 6), (4, 4, 8), (6, 4, 10), (6, 6, 11)] [1, 2, 3, 4, 5, 6, 7, 9, 8, 10, 11] [(2, 2, 4), (4, 2, 6), (4, 4, 9), (6, 4, 10), (6, 6, 11)]
[(2, 2, 4), (4, 2, 6), (4, 4,

[[],
 [],
 [],
 [],
 [],
 [([(2, 2, 4), (4, 2, 6), (4, 4, 8), (6, 4, 10), (6, 6, 11)],
   01 02 03 04 05 06 07 08 09 10 11
   02 04 01 06 03 08 05 10 07 11 09
   03 01 05 02 07 04 09 06 11 08 10
   04 06 02 08 01 10 03 11 05 09 07
   05 03 07 01 09 02 11 04 10 06 08
   06 08 04 10 02 11 01 09 03 07 05
   07 05 09 03 11 01 10 02 08 04 06
   08 10 06 11 04 09 02 07 01 05 03
   09 07 11 05 10 03 08 01 06 02 04
   10 11 08 09 06 07 04 05 02 03 01
   11 09 10 07 08 05 06 03 04 01 02)]]

[(2, 2, 4), (4, 2, 6), (4, 4, 8), (6, 4, 10), (6, 6, 11)] [1, 2, 3, 8, 9, 5, 4, 10, 11, 6, 7] [(2, 2, 8), (4, 2, 9), (4, 3, 11), (4, 4, 6), (6, 2, 7)]
[(2, 2, 4), (4, 2, 6), (4, 4, 8), (6, 4, 10), (6, 6, 11)] [1, 2, 3, 8, 9, 5, 4, 10, 11, 7, 6] [(2, 2, 8), (4, 2, 9), (4, 3, 11), (4, 4, 7), (6, 2, 11)]
[(2, 2, 4), (4, 2, 6), (4, 4, 8), (6, 4, 10), (6, 6, 11)] [1, 2, 3, 8, 9, 7, 6, 11, 10, 5, 4] [(2, 2, 8), (4, 2, 10), (4, 3, 5), (6, 2, 9), (6, 3, 10)]
[(2, 2, 4), (4, 2, 6), (4, 4, 8), (6, 4, 10), (6, 6, 11)] [1, 2, 3, 8, 9, 6, 7, 11, 10, 5, 4] [(2, 2, 8), (4, 2, 10), (4, 3, 5), (6, 2, 11), (6, 3, 8)]
[(2, 2, 4), (4, 2, 6), (4, 4, 8), (6, 4, 10), (6, 6, 11)] [1, 2, 3, 8, 9, 6, 7, 4, 5, 10, 11] [(2, 2, 8), (4, 2, 10), (4, 3, 6), (2, 4, 10), (4, 4, 7), (6, 3, 8)]
[(2, 2, 4), (4, 2, 6), (4, 4, 8), (6, 4, 10), (6, 6, 11)] [1, 2, 3, 8, 9, 11, 10, 5, 4, 7, 6] [(2, 2, 8), (4, 2, 10), (4, 3, 6), (2, 4, 10), (4, 4, 11), (6, 3, 7)]
[(2, 2, 4), (4, 2, 6), (4, 4, 8), (6, 4, 10), (6, 6, 11)] [1, 2, 3

[(2, 2, 4), (4, 2, 6), (4, 4, 8), (6, 4, 10), (6, 6, 11)] [1, 2, 3, 9, 8, 4, 5, 10, 11, 7, 6] [(2, 2, 9), (4, 2, 10), (4, 3, 9), (4, 4, 6), (6, 2, 11)]
[(2, 2, 4), (4, 2, 6), (4, 4, 8), (6, 4, 10), (6, 6, 11)] [1, 2, 3, 9, 8, 4, 5, 10, 11, 6, 7] [(2, 2, 9), (4, 2, 10), (4, 3, 9), (4, 4, 7), (6, 2, 7)]
[(2, 2, 4), (4, 2, 6), (4, 4, 8), (6, 4, 10), (6, 6, 11)] [1, 2, 3, 9, 8, 7, 6, 10, 11, 5, 4] [(2, 2, 9), (4, 2, 11), (4, 3, 5), (6, 2, 8), (6, 3, 11)]
[(2, 2, 4), (4, 2, 6), (4, 4, 8), (6, 4, 10), (6, 6, 11)] [1, 2, 3, 9, 8, 6, 7, 10, 11, 5, 4] [(2, 2, 9), (4, 2, 11), (4, 3, 5), (6, 2, 10), (6, 3, 9)]
[(2, 2, 4), (4, 2, 6), (4, 4, 8), (6, 4, 10), (6, 6, 11)] [1, 2, 3, 9, 8, 6, 7, 4, 5, 11, 10] [(2, 2, 9), (4, 2, 11), (4, 3, 6), (2, 4, 11), (4, 4, 7), (6, 3, 9)]
[(2, 2, 4), (4, 2, 6), (4, 4, 8), (6, 4, 10), (6, 6, 11)] [1, 2, 3, 9, 8, 10, 11, 5, 4, 7, 6] [(2, 2, 9), (4, 2, 11), (4, 3, 6), (2, 4, 11), (4, 4, 10), (6, 3, 7)]
[(2, 2, 4), (4, 2, 6), (4, 4, 8), (6, 4, 10), (6, 6, 11)] [1, 2, 3

[(2, 2, 4), (4, 2, 6), (4, 4, 8), (6, 4, 10), (6, 6, 11)] [1, 2, 3, 10, 11, 5, 4, 8, 9, 6, 7] [(2, 2, 10), (4, 2, 11), (4, 3, 9), (4, 4, 6), (6, 2, 7)]
[(2, 2, 4), (4, 2, 6), (4, 4, 8), (6, 4, 10), (6, 6, 11)] [1, 2, 3, 10, 11, 5, 4, 8, 9, 7, 6] [(2, 2, 10), (4, 2, 11), (4, 3, 9), (4, 4, 7), (6, 2, 9)]
[(2, 2, 4), (4, 2, 6), (4, 4, 8), (6, 4, 10), (6, 6, 11)] [1, 2, 3, 11, 10, 8, 9, 6, 7, 4, 5] [(2, 2, 11), (4, 2, 5), (4, 3, 6), (6, 3, 8), (6, 6, 9)]
[(2, 2, 4), (4, 2, 6), (4, 4, 8), (6, 4, 10), (6, 6, 11)] [1, 2, 3, 11, 10, 9, 8, 6, 7, 4, 5] [(2, 2, 11), (4, 2, 5), (4, 3, 6), (6, 3, 9), (6, 6, 8)]
[(2, 2, 4), (4, 2, 6), (4, 4, 8), (6, 4, 10), (6, 6, 11)] [1, 2, 3, 11, 10, 9, 8, 7, 6, 4, 5] [(2, 2, 11), (4, 2, 5), (4, 3, 7), (6, 2, 8), (6, 6, 9)]
[(2, 2, 4), (4, 2, 6), (4, 4, 8), (6, 4, 10), (6, 6, 11)] [1, 2, 3, 11, 10, 8, 9, 7, 6, 4, 5] [(2, 2, 11), (4, 2, 5), (4, 3, 7), (6, 2, 9), (6, 6, 8)]
[(2, 2, 4), (4, 2, 6), (4, 4, 8), (6, 4, 10), (6, 6, 11)] [1, 2, 3, 11, 10, 6, 7, 8, 9, 4, 5

[[],
 [],
 [],
 [],
 [],
 [([(2, 2, 4), (4, 2, 6), (4, 4, 8), (6, 4, 10), (6, 6, 11)],
   01 02 03 04 05 06 07 08 09 10 11
   02 04 01 06 03 08 05 10 07 11 09
   03 01 05 02 07 04 09 06 11 08 10
   04 06 02 08 01 10 03 11 05 09 07
   05 03 07 01 09 02 11 04 10 06 08
   06 08 04 10 02 11 01 09 03 07 05
   07 05 09 03 11 01 10 02 08 04 06
   08 10 06 11 04 09 02 07 01 05 03
   09 07 11 05 10 03 08 01 06 02 04
   10 11 08 09 06 07 04 05 02 03 01
   11 09 10 07 08 05 06 03 04 01 02)]]

In [121]:
G11 = Gs[5][0][1]

In [122]:
findMorphism(G11,G11,False)

{(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11),
 (1, 3, 2, 5, 4, 7, 6, 9, 8, 11, 10),
 (1, 4, 5, 8, 9, 11, 10, 7, 6, 3, 2),
 (1, 5, 4, 9, 8, 10, 11, 6, 7, 2, 3),
 (1, 6, 7, 11, 10, 5, 4, 2, 3, 8, 9),
 (1, 7, 6, 10, 11, 4, 5, 3, 2, 9, 8),
 (1, 8, 9, 7, 6, 2, 3, 10, 11, 5, 4),
 (1, 9, 8, 6, 7, 3, 2, 11, 10, 4, 5),
 (1, 10, 11, 3, 2, 8, 9, 5, 4, 6, 7),
 (1, 11, 10, 2, 3, 9, 8, 4, 5, 7, 6)}

# 12 : 5 (3)

## G12_4 = Dih6 = Dih3 x Z2  !!!

In [123]:
G12_4 = Group("""   01 02 03 04 05 06 07 08 09 10 11 12
   02 01 04 03 06 05 08 07 11 12 09 10
   03 04 01 02 10 12 09 11 07 05 08 06
   04 03 02 01 12 10 11 09 08 06 07 05
   05 06 09 11 01 02 10 12 03 07 04 08
   06 05 11 09 02 01 12 10 04 08 03 07
   07 08 10 12 09 11 01 02 05 03 06 04
   08 07 12 10 11 09 02 01 06 04 05 03
   09 11 05 06 07 08 03 04 10 01 12 02
   10 12 07 08 03 04 05 06 01 09 02 11
   11 09 06 05 08 07 04 03 12 02 10 01
   12 10 08 07 04 03 06 05 02 11 01 09""")
closes(G12_4)

{frozenset({1}),
 frozenset({1, 4}),
 frozenset({1, 7}),
 frozenset({1, 2}),
 frozenset({1, 9, 10}),
 frozenset({1, 2, 9, 10, 11, 12}),
 frozenset({1, 5}),
 frozenset({1, 8}),
 frozenset({1, 6}),
 frozenset({1, 3})}

## G12_5 = Z6 x Z2

In [124]:
G12_5 = Group("""   01 02 03 04 05 06 07 08 09 10 11 12
   02 01 04 03 07 08 05 06 11 12 09 10
   03 04 01 02 09 10 11 12 05 06 07 08
   04 03 02 01 11 12 09 10 07 08 05 06
   05 07 09 11 06 01 08 02 10 03 12 04
   06 08 10 12 01 05 02 07 03 09 04 11
   07 05 11 09 08 02 06 01 12 04 10 03
   08 06 12 10 02 07 01 05 04 11 03 09
   09 11 05 07 10 03 12 04 06 01 08 02
   10 12 06 08 03 09 04 11 01 05 02 07
   11 09 07 05 12 04 10 03 08 02 06 01
   12 10 08 06 04 11 03 09 02 07 01 05""")
closes(G12_5)

{frozenset({1}),
 frozenset({1, 4}),
 frozenset({1, 2}),
 frozenset({1, 5, 6}),
 frozenset({1, 2, 5, 6, 7, 8}),
 frozenset({1, 4, 5, 6, 11, 12}),
 frozenset({1, 3}),
 frozenset({1, 3, 5, 6, 9, 10})}

## G12_3 = (Z2 x Z2)x|Z3  !!!

In [125]:
G12_3 = Group("""   01 02 03 04 05 06 07 08 09 10 11 12
   02 01 04 03 09 08 11 06 05 12 07 10
   03 04 01 02 11 10 09 12 07 06 05 08
   04 03 02 01 07 12 05 10 11 08 09 06
   05 07 09 11 06 01 10 04 12 02 08 03
   06 10 12 08 01 05 02 11 03 07 04 09
   07 05 11 09 12 04 08 01 06 03 10 02
   08 12 10 06 02 09 01 07 04 11 03 05
   09 11 05 07 08 02 12 03 10 01 06 04
   10 06 08 12 03 11 04 05 01 09 02 07
   11 09 07 05 10 03 06 02 08 04 12 01
   12 08 06 10 04 07 03 09 02 05 01 11""")
closes(G12_3)

{frozenset({1}),
 frozenset({1, 4}),
 frozenset({1, 2}),
 frozenset({1, 9, 10}),
 frozenset({1, 7, 8}),
 frozenset({1, 11, 12}),
 frozenset({1, 3}),
 frozenset({1, 5, 6})}

## G12_1 = Dic3 = Z3 x| Z4  !!!

In [126]:
G12_1 = Group("""   01 02 03 04 05 06 07 08 09 10 11 12
   02 01 04 03 06 05 09 10 07 08 12 11
   03 04 02 01 08 10 11 06 12 05 09 07
   04 03 01 02 10 08 12 05 11 06 07 09
   05 06 07 09 02 01 04 11 03 12 10 08
   06 05 09 07 01 02 03 12 04 11 08 10
   07 09 06 05 11 12 10 01 08 02 03 04
   08 10 11 12 04 03 01 09 02 07 05 06
   09 07 05 06 12 11 08 02 10 01 04 03
   10 08 12 11 03 04 02 07 01 09 06 05
   11 12 10 08 09 07 05 03 06 04 02 01
   12 11 08 10 07 09 06 04 05 03 01 02""")
closes(G12_1)

{frozenset({1}),
 frozenset({1, 2}),
 frozenset({1, 2, 5, 6}),
 frozenset({1, 9, 10}),
 frozenset({1, 2, 3, 4}),
 frozenset({1, 2, 11, 12}),
 frozenset({1, 2, 7, 8, 9, 10})}

## G12_2 = Z12 = Z4 x Z3

In [127]:
G12_2 = Group("""   01 02 03 04 05 06 07 08 09 10 11 12
   02 01 04 03 07 08 05 06 11 12 09 10
   03 04 02 01 09 12 11 10 07 06 05 08
   04 03 01 02 11 10 09 12 05 08 07 06
   05 07 09 11 06 01 08 02 12 04 10 03
   06 08 12 10 01 05 02 07 03 11 04 09
   07 05 11 09 08 02 06 01 10 03 12 04
   08 06 10 12 02 07 01 05 04 09 03 11
   09 11 07 05 12 03 10 04 08 01 06 02
   10 12 06 08 04 11 03 09 01 07 02 05
   11 09 05 07 10 04 12 03 06 02 08 01
   12 10 08 06 03 09 04 11 02 05 01 07""")
closes(G12_2)

{frozenset({1}),
 frozenset({1, 2}),
 frozenset({1, 5, 6}),
 frozenset({1, 2, 5, 6, 7, 8}),
 frozenset({1, 2, 3, 4}),
 frozenset({1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12})}

## прочее

In [129]:
if 0:
    N=12
    DEBUG = True
    VERBOSE = False
    Gs = []
    for j in range((N-1)//2+1):
        p=diaperm(N,j)
        print('perm:',j,p)
        G = Group(N,p)
        conc = {}
        if not startCheck(G,conc): continue
        if len(conc)>0 : print(conc)
        Gss = []
        for GG in fastGen(G):
            notNew = False
            for GGG in Gss:
                m = findMorphism(GGG[1],GG[1],True)
                if m:
                    notNew = True
                    print(GGG[0],m,GG[0])
                    break
            if not notNew:
                Gss.append(GG)
                print(GG[0])
                print(GG[1])
        Gs.append(Gss)
    Gs
    # executed in 10m 55s, finished 20:53:45 2023-02-01
    # executed in 24m 24s, finished 11:26:23 2023-02-02

[(3, 2, 4), (5, 2, 6), (5, 3, 9), (5, 4, 11), (7, 2, 8), (7, 3, 10), (7, 5, 9)] [1, 2, 3, 5, 4, 8, 6, 7, 9, 10, 12, 11] [(3, 2, 5), (4, 2, 8), (4, 3, 9), (5, 4, 11), (6, 2, 7), (6, 3, 10), (6, 4, 9)]
[(3, 2, 4), (5, 2, 6), (5, 3, 9), (5, 4, 11), (7, 2, 8), (7, 3, 10), (7, 5, 9)] [1, 2, 3, 5, 7, 6, 8, 4, 11, 12, 10, 9] [(3, 2, 5), (4, 2, 8), (4, 3, 9), (5, 4, 11), (6, 2, 7), (6, 3, 10), (6, 4, 12)]
[(3, 2, 4), (5, 2, 6), (5, 3, 9), (5, 4, 11), (7, 2, 8), (7, 3, 10), (7, 5, 9)] [1, 2, 3, 5, 6, 7, 8, 4, 11, 12, 10, 9] [(3, 2, 5), (4, 2, 8), (4, 3, 9), (5, 4, 11), (6, 2, 7), (6, 3, 11), (6, 4, 9)]
[(3, 2, 4), (5, 2, 6), (5, 3, 9), (5, 4, 11), (7, 2, 8), (7, 3, 10), (7, 5, 9)] [1, 2, 3, 5, 4, 8, 7, 6, 9, 10, 12, 11] [(3, 2, 5), (4, 2, 8), (4, 3, 9), (5, 4, 11), (6, 2, 7), (6, 3, 11), (6, 4, 12)]
[(3, 2, 4), (5, 2, 6), (5, 3, 9), (5, 4, 11), (7, 2, 8), (7, 3, 10), (7, 5, 9)] [1, 2, 3, 5, 4, 8, 6, 7, 9, 10, 11, 12] [(3, 2, 5), (4, 2, 8), (4, 3, 9), (5, 4, 12), (6, 2, 7), (6, 3, 10), (6, 4, 9)

[(3, 2, 4), (5, 2, 6), (5, 3, 9), (5, 4, 11), (7, 2, 8), (7, 3, 10), (7, 5, 9)] [1, 3, 2, 5, 4, 7, 8, 6, 9, 10, 11, 12] [(3, 2, 5), (4, 2, 9), (4, 3, 7), (5, 4, 12), (6, 2, 12), (6, 3, 8), (6, 4, 11)]
[(3, 2, 4), (5, 2, 6), (5, 3, 9), (5, 4, 11), (7, 2, 8), (7, 3, 10), (7, 5, 9)] [1, 3, 2, 5, 4, 8, 6, 7, 9, 10, 12, 11] [(3, 2, 5), (4, 2, 9), (4, 3, 8), (5, 4, 11), (6, 2, 10), (6, 3, 7), (6, 4, 9)]
[(3, 2, 4), (5, 2, 6), (5, 3, 9), (5, 4, 11), (7, 2, 8), (7, 3, 10), (7, 5, 9)] [1, 3, 2, 5, 7, 6, 8, 4, 11, 12, 10, 9] [(3, 2, 5), (4, 2, 9), (4, 3, 8), (5, 4, 11), (6, 2, 10), (6, 3, 7), (6, 4, 12)]
[(3, 2, 4), (5, 2, 6), (5, 3, 9), (5, 4, 11), (7, 2, 8), (7, 3, 10), (7, 5, 9)] [1, 3, 2, 5, 6, 7, 8, 4, 11, 12, 10, 9] [(3, 2, 5), (4, 2, 9), (4, 3, 8), (5, 4, 11), (6, 2, 11), (6, 3, 7), (6, 4, 9)]
[(3, 2, 4), (5, 2, 6), (5, 3, 9), (5, 4, 11), (7, 2, 8), (7, 3, 10), (7, 5, 9)] [1, 3, 2, 5, 4, 8, 7, 6, 9, 10, 12, 11] [(3, 2, 5), (4, 2, 9), (4, 3, 8), (5, 4, 11), (6, 2, 11), (6, 3, 7), (6, 4, 12

[(3, 2, 4), (5, 2, 6), (5, 3, 9), (5, 4, 11), (7, 2, 8), (7, 3, 10), (7, 5, 9)] [1, 3, 2, 5, 4, 7, 6, 8, 10, 9, 12, 11] [(3, 2, 5), (4, 2, 10), (4, 3, 7), (5, 4, 11), (6, 2, 9), (6, 3, 8), (6, 4, 10)]
[(3, 2, 4), (5, 2, 6), (5, 3, 9), (5, 4, 11), (7, 2, 8), (7, 3, 10), (7, 5, 9)] [1, 3, 2, 5, 7, 4, 8, 6, 12, 11, 10, 9] [(3, 2, 5), (4, 2, 10), (4, 3, 7), (5, 4, 11), (6, 2, 9), (6, 3, 8), (6, 4, 12)]
[(3, 2, 4), (5, 2, 6), (5, 3, 9), (5, 4, 11), (7, 2, 8), (7, 3, 10), (7, 5, 9)] [1, 3, 2, 5, 6, 8, 7, 4, 11, 12, 9, 10] [(3, 2, 5), (4, 2, 10), (4, 3, 7), (5, 4, 11), (6, 2, 11), (6, 3, 8), (6, 4, 10)]
[(3, 2, 4), (5, 2, 6), (5, 3, 9), (5, 4, 11), (7, 2, 8), (7, 3, 10), (7, 5, 9)] [1, 3, 2, 5, 4, 7, 8, 6, 10, 9, 12, 11] [(3, 2, 5), (4, 2, 10), (4, 3, 7), (5, 4, 11), (6, 2, 11), (6, 3, 8), (6, 4, 12)]
[(3, 2, 4), (5, 2, 6), (5, 3, 9), (5, 4, 11), (7, 2, 8), (7, 3, 10), (7, 5, 9)] [1, 3, 2, 5, 4, 7, 6, 8, 10, 9, 11, 12] [(3, 2, 5), (4, 2, 10), (4, 3, 7), (5, 4, 12), (6, 2, 9), (6, 3, 8), (6, 4

[(3, 2, 4), (5, 2, 6), (5, 3, 9), (5, 4, 11), (7, 2, 8), (7, 3, 10), (7, 5, 9)] [1, 3, 2, 5, 6, 4, 8, 7, 10, 9, 11, 12] [(3, 2, 5), (4, 2, 11), (4, 3, 6), (5, 4, 9), (7, 2, 12), (7, 3, 8), (7, 4, 10)]
[(3, 2, 4), (5, 2, 6), (5, 3, 9), (5, 4, 11), (7, 2, 8), (7, 3, 10), (7, 5, 9)] [1, 3, 2, 5, 4, 6, 7, 8, 11, 12, 10, 9] [(3, 2, 5), (4, 2, 11), (4, 3, 6), (5, 4, 9), (7, 2, 12), (7, 3, 8), (7, 4, 11)]
[(3, 2, 4), (5, 2, 6), (5, 3, 9), (5, 4, 11), (7, 2, 8), (7, 3, 10), (7, 5, 9)] [1, 3, 2, 5, 4, 6, 8, 7, 11, 12, 9, 10] [(3, 2, 5), (4, 2, 11), (4, 3, 6), (5, 4, 10), (7, 2, 10), (7, 3, 8), (7, 4, 9)]
[(3, 2, 4), (5, 2, 6), (5, 3, 9), (5, 4, 11), (7, 2, 8), (7, 3, 10), (7, 5, 9)] [1, 3, 2, 5, 6, 4, 7, 8, 9, 10, 11, 12] [(3, 2, 5), (4, 2, 11), (4, 3, 6), (5, 4, 10), (7, 2, 10), (7, 3, 8), (7, 4, 11)]
[(3, 2, 4), (5, 2, 6), (5, 3, 9), (5, 4, 11), (7, 2, 8), (7, 3, 10), (7, 5, 9)] [1, 3, 2, 5, 6, 4, 8, 7, 9, 10, 11, 12] [(3, 2, 5), (4, 2, 11), (4, 3, 6), (5, 4, 10), (7, 2, 12), (7, 3, 8), (7, 4

[(3, 2, 4), (5, 2, 6), (5, 3, 9), (5, 4, 11), (7, 2, 8), (7, 3, 10), (7, 5, 9)] [1, 5, 2, 3, 4, 8, 6, 7, 11, 12, 10, 9] [(3, 2, 5), (4, 2, 11), (4, 3, 10), (5, 4, 8), (6, 2, 12), (6, 4, 11), (6, 5, 7)]
[(3, 2, 4), (5, 2, 6), (5, 3, 9), (5, 4, 11), (7, 2, 8), (7, 3, 10), (7, 5, 9)] [1, 3, 2, 5, 4, 6, 8, 7, 12, 11, 10, 9] [(3, 2, 5), (4, 2, 12), (4, 3, 6), (5, 4, 9), (7, 2, 9), (7, 3, 8), (7, 4, 10)]
[(3, 2, 4), (5, 2, 6), (5, 3, 9), (5, 4, 11), (7, 2, 8), (7, 3, 10), (7, 5, 9)] [1, 3, 2, 5, 6, 4, 7, 8, 10, 9, 12, 11] [(3, 2, 5), (4, 2, 12), (4, 3, 6), (5, 4, 9), (7, 2, 9), (7, 3, 8), (7, 4, 12)]
[(3, 2, 4), (5, 2, 6), (5, 3, 9), (5, 4, 11), (7, 2, 8), (7, 3, 10), (7, 5, 9)] [1, 3, 2, 5, 6, 4, 8, 7, 10, 9, 12, 11] [(3, 2, 5), (4, 2, 12), (4, 3, 6), (5, 4, 9), (7, 2, 11), (7, 3, 8), (7, 4, 10)]
[(3, 2, 4), (5, 2, 6), (5, 3, 9), (5, 4, 11), (7, 2, 8), (7, 3, 10), (7, 5, 9)] [1, 3, 2, 5, 4, 6, 7, 8, 12, 11, 10, 9] [(3, 2, 5), (4, 2, 12), (4, 3, 6), (5, 4, 9), (7, 2, 11), (7, 3, 8), (7, 4, 1

...

[(3, 2, 4), (5, 2, 7), (5, 3, 9), (5, 4, 11), (2, 5, 7), (3, 5, 9), (5, 5, 6)] [1, 2, 5, 6, 9, 10, 8, 7, 12, 11, 4, 3] [(3, 2, 11), (2, 3, 11), (3, 3, 8), (5, 2, 6), (5, 3, 7), (5, 4, 9)]
[(3, 2, 4), (5, 2, 7), (5, 3, 9), (5, 4, 11), (2, 5, 7), (3, 5, 9), (5, 5, 6)] [1, 2, 5, 6, 9, 10, 7, 8, 3, 4, 11, 12] [(3, 2, 11), (2, 3, 11), (3, 3, 8), (5, 2, 6), (5, 3, 7), (5, 4, 10)]
[(3, 2, 4), (5, 2, 7), (5, 3, 9), (5, 4, 11), (2, 5, 7), (3, 5, 9), (5, 5, 6)] [1, 2, 5, 6, 9, 10, 7, 8, 11, 12, 3, 4] [(3, 2, 11), (2, 3, 11), (3, 3, 8), (5, 2, 6), (5, 3, 9), (5, 4, 8)]
[(3, 2, 4), (5, 2, 7), (5, 3, 9), (5, 4, 11), (2, 5, 7), (3, 5, 9), (5, 5, 6)] [1, 2, 5, 6, 9, 10, 8, 7, 4, 3, 12, 11] [(3, 2, 11), (2, 3, 11), (3, 3, 8), (5, 2, 6), (5, 3, 10), (5, 4, 8)]
[(3, 2, 4), (5, 2, 7), (5, 3, 9), (5, 4, 11), (2, 5, 7), (3, 5, 9), (5, 5, 6)] [1, 2, 9, 10, 5, 6, 7, 8, 11, 12, 3, 4] [(3, 2, 11), (2, 3, 11), (3, 3, 8), (5, 2, 7), (5, 4, 9), (5, 5, 6)]
[(3, 2, 4), (5, 2, 7), (5, 3, 9), (5, 4, 11), (2, 5, 7), (

[(3, 2, 4), (5, 2, 7), (5, 3, 9), (5, 4, 11), (2, 5, 7), (3, 5, 9), (5, 5, 6)] [1, 2, 9, 10, 11, 12, 3, 4, 6, 5, 7, 8] [(3, 2, 11), (2, 3, 11), (3, 3, 12), (5, 2, 8), (5, 3, 9), (5, 4, 6)]
[(3, 2, 4), (5, 2, 7), (5, 3, 9), (5, 4, 11), (2, 5, 7), (3, 5, 9), (5, 5, 6)] [1, 2, 9, 10, 11, 12, 3, 4, 7, 8, 6, 5] [(3, 2, 11), (2, 3, 11), (3, 3, 12), (5, 2, 8), (5, 3, 10), (5, 4, 6)]
[(3, 2, 4), (5, 2, 7), (5, 3, 9), (5, 4, 11), (2, 5, 7), (3, 5, 9), (5, 5, 6)] [1, 2, 7, 8, 11, 12, 3, 4, 9, 10, 5, 6] [(3, 2, 11), (2, 3, 11), (3, 3, 12), (5, 2, 9), (5, 3, 6), (5, 4, 7)]
[(3, 2, 4), (5, 2, 7), (5, 3, 9), (5, 4, 11), (2, 5, 7), (3, 5, 9), (5, 5, 6)] [1, 2, 7, 8, 11, 12, 3, 4, 5, 6, 9, 10] [(3, 2, 11), (2, 3, 11), (3, 3, 12), (5, 2, 9), (5, 3, 6), (5, 4, 8)]
[(3, 2, 4), (5, 2, 7), (5, 3, 9), (5, 4, 11), (2, 5, 7), (3, 5, 9), (5, 5, 6)] [1, 2, 7, 8, 11, 12, 3, 4, 6, 5, 10, 9] [(3, 2, 11), (2, 3, 11), (3, 3, 12), (5, 2, 9), (5, 3, 7), (5, 4, 6)]
[(3, 2, 4), (5, 2, 7), (5, 3, 9), (5, 4, 11), (2, 5, 7

[(3, 2, 4), (5, 2, 7), (5, 3, 9), (5, 4, 11), (2, 5, 7), (3, 5, 9), (5, 5, 6)] [1, 2, 7, 8, 9, 10, 5, 6, 4, 3, 11, 12] [(3, 2, 12), (2, 3, 12), (3, 3, 5), (5, 2, 9), (5, 3, 7), (5, 5, 10)]
[(3, 2, 4), (5, 2, 7), (5, 3, 9), (5, 4, 11), (2, 5, 7), (3, 5, 9), (5, 5, 6)] [1, 2, 7, 8, 9, 10, 5, 6, 11, 12, 4, 3] [(3, 2, 12), (2, 3, 12), (3, 3, 5), (5, 2, 9), (5, 3, 8), (5, 5, 10)]
[(3, 2, 4), (5, 2, 7), (5, 3, 9), (5, 4, 11), (2, 5, 7), (3, 5, 9), (5, 5, 6)] [1, 2, 7, 8, 9, 10, 6, 5, 12, 11, 3, 4] [(3, 2, 12), (2, 3, 12), (3, 3, 5), (5, 2, 10), (5, 3, 7), (5, 5, 9)]
[(3, 2, 4), (5, 2, 7), (5, 3, 9), (5, 4, 11), (2, 5, 7), (3, 5, 9), (5, 5, 6)] [1, 2, 7, 8, 9, 10, 6, 5, 3, 4, 12, 11] [(3, 2, 12), (2, 3, 12), (3, 3, 5), (5, 2, 10), (5, 3, 8), (5, 5, 9)]
[(3, 2, 4), (5, 2, 7), (5, 3, 9), (5, 4, 11), (2, 5, 7), (3, 5, 9), (5, 5, 6)] [1, 2, 9, 10, 7, 8, 5, 6, 3, 4, 12, 11] [(3, 2, 12), (2, 3, 12), (3, 3, 6), (5, 2, 7), (5, 4, 9), (5, 5, 8)]
[(3, 2, 4), (5, 2, 7), (5, 3, 9), (5, 4, 11), (2, 5, 7),

[(3, 2, 4), (5, 2, 7), (5, 3, 9), (5, 4, 11), (2, 5, 7), (3, 5, 9), (5, 5, 6)] [1, 2, 5, 6, 11, 12, 4, 3, 8, 7, 9, 10] [(3, 2, 12), (2, 3, 12), (3, 3, 11), (5, 2, 6), (5, 3, 7), (5, 4, 9), (3, 5, 7)]
[(3, 2, 4), (5, 2, 6), (5, 3, 7), (5, 4, 9), (7, 5, 11), (3, 7, 11)] [1, 2, 5, 6, 8, 7, 4, 3, 11, 12, 9, 10] [(3, 2, 12), (2, 3, 12), (3, 3, 11), (5, 2, 6), (5, 3, 7), (5, 4, 9), (3, 5, 9)]
[(3, 2, 4), (5, 2, 7), (5, 3, 9), (5, 4, 11), (2, 5, 7), (3, 5, 9), (5, 5, 6)] [1, 2, 5, 6, 11, 12, 4, 3, 8, 7, 10, 9] [(3, 2, 12), (2, 3, 12), (3, 3, 11), (5, 2, 6), (5, 3, 7), (5, 4, 10), (3, 5, 7)]
[(3, 2, 4), (5, 2, 6), (5, 3, 7), (5, 4, 9), (7, 5, 11), (3, 7, 11)] [1, 2, 5, 6, 8, 7, 4, 3, 11, 12, 10, 9] [(3, 2, 12), (2, 3, 12), (3, 3, 11), (5, 2, 6), (5, 3, 7), (5, 4, 10), (3, 5, 10)]
[(3, 2, 4), (5, 2, 7), (5, 3, 9), (5, 4, 11), (2, 5, 7), (3, 5, 9), (5, 5, 6)] [1, 2, 5, 6, 11, 12, 4, 3, 7, 8, 9, 10] [(3, 2, 12), (2, 3, 12), (3, 3, 11), (5, 2, 6), (5, 3, 8), (5, 4, 9), (3, 5, 8)]
[(3, 2, 4), (5, 2

[[],
 [],
 [([(3, 2, 4),
    (5, 2, 6),
    (5, 3, 9),
    (5, 4, 11),
    (7, 2, 8),
    (7, 3, 10),
    (7, 5, 9)],
   01 02 03 04 05 06 07 08 09 10 11 12
   02 01 04 03 06 05 08 07 11 12 09 10
   03 04 01 02 10 12 09 11 07 05 08 06
   04 03 02 01 12 10 11 09 08 06 07 05
   05 06 09 11 01 02 10 12 03 07 04 08
   06 05 11 09 02 01 12 10 04 08 03 07
   07 08 10 12 09 11 01 02 05 03 06 04
   08 07 12 10 11 09 02 01 06 04 05 03
   09 11 05 06 07 08 03 04 10 01 12 02
   10 12 07 08 03 04 05 06 01 09 02 11
   11 09 06 05 08 07 04 03 12 02 10 01
   12 10 08 07 04 03 06 05 02 11 01 09)],
 [],
 [([(3, 2, 4),
    (5, 2, 7),
    (5, 3, 9),
    (5, 4, 11),
    (2, 5, 7),
    (3, 5, 9),
    (5, 5, 6)],
   01 02 03 04 05 06 07 08 09 10 11 12
   02 01 04 03 07 08 05 06 11 12 09 10
   03 04 01 02 09 10 11 12 05 06 07 08
   04 03 02 01 11 12 09 10 07 08 05 06
   05 07 09 11 06 01 08 02 10 03 12 04
   06 08 10 12 01 05 02 07 03 09 04 11
   07 05 11 09 08 02 06 01 12 04 10 03
   08 06 12 10 02 07 01 05

# 13 : 1

## G13_1 = Z13

In [130]:
G13_1 = Group("""   01 02 03 04 05 06 07 08 09 10 11 12 13
   02 04 01 06 03 08 05 10 07 12 09 13 11
   03 01 05 02 07 04 09 06 11 08 13 10 12
   04 06 02 08 01 10 03 12 05 13 07 11 09
   05 03 07 01 09 02 11 04 13 06 12 08 10
   06 08 04 10 02 12 01 13 03 11 05 09 07
   07 05 09 03 11 01 13 02 12 04 10 06 08
   08 10 06 12 04 13 02 11 01 09 03 07 05
   09 07 11 05 13 03 12 01 10 02 08 04 06
   10 12 08 13 06 11 04 09 02 07 01 05 03
   11 09 13 07 12 05 10 03 08 01 06 02 04
   12 13 10 11 08 09 06 07 04 05 02 03 01
   13 11 12 09 10 07 08 05 06 03 04 01 02""")
closes(G13_1)

{frozenset({1}), frozenset({1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13})}

# 14 : 2 (1)

## G14_1 = Dih7 !!!

In [131]:
G14_1 = Group("""01 02 03 04 05 06 07 08 09 10 11 12 13 14
02 01 10 09 12 11 14 13 04 03 06 05 08 07
03 09 01 12 14 10 13 11 02 06 08 04 07 05
04 10 11 01 09 13 12 14 05 02 03 07 06 08
05 11 13 10 01 14 09 12 07 04 02 08 03 06
06 12 09 14 13 01 11 10 03 08 07 02 05 04
07 13 14 11 10 12 01 09 08 05 04 06 02 03
08 14 12 13 11 09 10 01 06 07 05 03 04 02
09 03 06 02 04 08 05 07 12 01 10 14 11 13
10 04 02 05 07 03 08 06 01 11 13 09 14 12
11 05 04 07 08 02 06 03 10 13 14 01 12 09
12 06 08 03 02 07 04 05 14 09 01 13 10 11
13 07 05 08 06 04 03 02 11 14 12 10 09 01
14 08 07 06 03 05 02 04 13 12 09 11 01 10""")
closes(G14_1)

{frozenset({1}),
 frozenset({1, 4}),
 frozenset({1, 7}),
 frozenset({1, 2}),
 frozenset({1, 5}),
 frozenset({1, 8}),
 frozenset({1, 9, 10, 11, 12, 13, 14}),
 frozenset({1, 6}),
 frozenset({1, 3})}

## G14_2 = Z14 = Z7 x Z2

In [132]:
G14_2 = Group("""01 02 03 04 05 06 07 08 09 10 11 12 13 14
02 01 05 06 03 04 09 10 07 08 13 14 11 12
03 05 07 01 09 02 11 04 13 06 12 08 14 10
04 06 01 08 02 10 03 12 05 14 07 11 09 13
05 03 09 02 07 01 13 06 11 04 14 10 12 08
06 04 02 10 01 08 05 14 03 12 09 13 07 11
07 09 11 03 13 05 12 01 14 02 08 04 10 06
08 10 04 12 06 14 01 11 02 13 03 07 05 09
09 07 13 05 11 03 14 02 12 01 10 06 08 04
10 08 06 14 04 12 02 13 01 11 05 09 03 07
11 13 12 07 14 09 08 03 10 05 04 01 06 02
12 14 08 11 10 13 04 07 06 09 01 03 02 05
13 11 14 09 12 07 10 05 08 03 06 02 04 01
14 12 10 13 08 11 06 09 04 07 02 05 01 03""")
closes(G14_2)

{frozenset({1}),
 frozenset({1, 3, 4, 7, 8, 11, 12}),
 frozenset({1, 2}),
 frozenset({1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14})}

# 15 : 1

## G15_1 = Z15 = Z5 x Z3

In [133]:
G15_1 = Group("""01 02 03 04 05 06 07 08 09 10 11 12 13 14 15
02 03 01 06 09 08 05 04 07 12 15 14 11 10 13
03 01 02 08 07 04 09 06 05 14 13 10 15 12 11
04 06 08 10 01 12 03 14 02 11 05 15 07 13 09
05 09 07 01 11 02 13 03 15 04 10 06 14 08 12
06 08 04 12 02 14 01 10 03 15 09 13 05 11 07
07 05 09 03 13 01 15 02 11 08 14 04 12 06 10
08 04 06 14 03 10 02 12 01 13 07 11 09 15 05
09 07 05 02 15 03 11 01 13 06 12 08 10 04 14
10 12 14 11 04 15 08 13 06 05 01 09 03 07 02
11 15 13 05 10 09 14 07 12 01 04 02 08 03 06
12 14 10 15 06 13 04 11 08 09 02 07 01 05 03
13 11 15 07 14 05 12 09 10 03 08 01 06 02 04
14 10 12 13 08 11 06 15 04 07 03 05 02 09 01
15 13 11 09 12 07 10 05 14 02 06 03 04 01 08""")
closes(G15_1)

{frozenset({1}),
 frozenset({1, 2, 3}),
 frozenset({1, 4, 5, 10, 11}),
 frozenset({1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15})}

# 16 : 14 (9)

совпадают графы циклов у G16_5 и G16_6, а также у G16_10 и G16_13

## G16_14 = Z2 x Z2 x Z2 x Z2

In [134]:
G16_14 = Group("""01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16
02 01 04 03 06 05 08 07 10 09 12 11 14 13 16 15
03 04 01 02 07 08 05 06 11 12 09 10 15 16 13 14
04 03 02 01 08 07 06 05 12 11 10 09 16 15 14 13
05 06 07 08 01 02 03 04 13 14 15 16 09 10 11 12
06 05 08 07 02 01 04 03 14 13 16 15 10 09 12 11
07 08 05 06 03 04 01 02 15 16 13 14 11 12 09 10
08 07 06 05 04 03 02 01 16 15 14 13 12 11 10 09
09 10 11 12 13 14 15 16 01 02 03 04 05 06 07 08
10 09 12 11 14 13 16 15 02 01 04 03 06 05 08 07
11 12 09 10 15 16 13 14 03 04 01 02 07 08 05 06
12 11 10 09 16 15 14 13 04 03 02 01 08 07 06 05
13 14 15 16 09 10 11 12 05 06 07 08 01 02 03 04
14 13 16 15 10 09 12 11 06 05 08 07 02 01 04 03
15 16 13 14 11 12 09 10 07 08 05 06 03 04 01 02
16 15 14 13 12 11 10 09 08 07 06 05 04 03 02 01""")
closes(G16_14)

{frozenset({1}),
 frozenset({1, 14}),
 frozenset({1, 4}),
 frozenset({1, 9}),
 frozenset({1, 7}),
 frozenset({1, 15}),
 frozenset({1, 2}),
 frozenset({1, 11}),
 frozenset({1, 10}),
 frozenset({1, 12}),
 frozenset({1, 5}),
 frozenset({1, 8}),
 frozenset({1, 16}),
 frozenset({1, 6}),
 frozenset({1, 3}),
 frozenset({1, 13})}

## G16_11 = Dih4 x Z2 !!!

In [135]:
G16_14 = Group("""01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16
02 01 04 03 06 05 08 07 10 09 12 11 14 13 16 15
03 04 01 02 07 08 05 06 11 12 09 10 15 16 13 14
04 03 02 01 08 07 06 05 12 11 10 09 16 15 14 13
05 06 07 08 01 02 03 04 14 13 16 15 10 09 12 11
06 05 08 07 02 01 04 03 13 14 15 16 09 10 11 12
07 08 05 06 03 04 01 02 16 15 14 13 12 11 10 09
08 07 06 05 04 03 02 01 15 16 13 14 11 12 09 10
09 10 11 12 13 14 15 16 01 02 03 04 05 06 07 08
10 09 12 11 14 13 16 15 02 01 04 03 06 05 08 07
11 12 09 10 15 16 13 14 03 04 01 02 07 08 05 06
12 11 10 09 16 15 14 13 04 03 02 01 08 07 06 05
13 14 15 16 09 10 11 12 06 05 08 07 02 01 04 03
14 13 16 15 10 09 12 11 05 06 07 08 01 02 03 04
15 16 13 14 11 12 09 10 08 07 06 05 04 03 02 01
16 15 14 13 12 11 10 09 07 08 05 06 03 04 01 02""")
closes(G16_14)

{frozenset({1}),
 frozenset({1, 4}),
 frozenset({1, 9}),
 frozenset({1, 7}),
 frozenset({1, 2}),
 frozenset({1, 2, 15, 16}),
 frozenset({1, 11}),
 frozenset({1, 10}),
 frozenset({1, 12}),
 frozenset({1, 5}),
 frozenset({1, 8}),
 frozenset({1, 6}),
 frozenset({1, 3}),
 frozenset({1, 2, 13, 14})}

## G16_7 = Dih8 !!!

In [136]:
G16_7 = Group("""01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16
02 01 04 03 06 05 08 07 10 09 12 11 15 16 13 14
03 04 01 02 12 11 14 16 13 15 06 05 09 07 10 08
04 03 02 01 11 12 16 14 15 13 05 06 10 08 09 07
05 06 11 12 01 02 13 15 16 14 03 04 07 10 08 09
06 05 12 11 02 01 15 13 14 16 04 03 08 09 07 10
07 08 13 15 14 16 01 02 11 12 09 10 03 05 04 06
08 07 15 13 16 14 02 01 12 11 10 09 04 06 03 05
09 10 14 16 15 13 12 11 01 02 08 07 06 03 05 04
10 09 16 14 13 15 11 12 02 01 07 08 05 04 06 03
11 12 05 06 04 03 10 09 07 08 02 01 16 13 14 15
12 11 06 05 03 04 09 10 08 07 01 02 14 15 16 13
13 15 07 08 10 09 05 06 03 04 16 14 11 01 12 02
14 16 09 10 07 08 03 04 06 05 13 15 01 12 02 11
15 13 08 07 09 10 06 05 04 03 14 16 12 02 11 01
16 14 10 09 08 07 04 03 05 06 15 13 02 11 01 12""")
closes(G16_7)

{frozenset({1}),
 frozenset({1, 4}),
 frozenset({1, 9}),
 frozenset({1, 7}),
 frozenset({1, 2}),
 frozenset({1, 10}),
 frozenset({1, 5}),
 frozenset({1, 8}),
 frozenset({1, 6}),
 frozenset({1, 2, 11, 12}),
 frozenset({1, 3}),
 frozenset({1, 2, 11, 12, 13, 14, 15, 16})}

## G16_10 = Z4 x Z2 x Z2

In [137]:
G16_10 = Group("""01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16
02 01 04 03 06 05 08 07 10 09 12 11 14 13 16 15
03 04 01 02 07 08 05 06 11 12 09 10 15 16 13 14
04 03 02 01 08 07 06 05 12 11 10 09 16 15 14 13
05 06 07 08 01 02 03 04 13 14 15 16 09 10 11 12
06 05 08 07 02 01 04 03 14 13 16 15 10 09 12 11
07 08 05 06 03 04 01 02 15 16 13 14 11 12 09 10
08 07 06 05 04 03 02 01 16 15 14 13 12 11 10 09
09 10 11 12 13 14 15 16 02 01 04 03 06 05 08 07
10 09 12 11 14 13 16 15 01 02 03 04 05 06 07 08
11 12 09 10 15 16 13 14 04 03 02 01 08 07 06 05
12 11 10 09 16 15 14 13 03 04 01 02 07 08 05 06
13 14 15 16 09 10 11 12 06 05 08 07 02 01 04 03
14 13 16 15 10 09 12 11 05 06 07 08 01 02 03 04
15 16 13 14 11 12 09 10 08 07 06 05 04 03 02 01
16 15 14 13 12 11 10 09 07 08 05 06 03 04 01 02""")
closes(G16_10)

{frozenset({1}),
 frozenset({1, 4}),
 frozenset({1, 2, 15, 16}),
 frozenset({1, 7}),
 frozenset({1, 2}),
 frozenset({1, 5}),
 frozenset({1, 8}),
 frozenset({1, 2, 11, 12}),
 frozenset({1, 6}),
 frozenset({1, 2, 9, 10}),
 frozenset({1, 3}),
 frozenset({1, 2, 13, 14})}

## G16_3 = (Z2 x Z2) x| Z4 !!!

In [138]:
G16_3 = Group("""01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16
02 01 04 03 06 05 08 07 10 09 12 11 15 16 13 14
03 04 01 02 07 08 05 06 11 12 09 10 14 13 16 15
04 03 02 01 08 07 06 05 12 11 10 09 16 15 14 13
05 06 07 08 01 02 03 04 16 14 15 13 12 10 11 09
06 05 08 07 02 01 04 03 14 16 13 15 11 09 12 10
07 08 05 06 03 04 01 02 15 13 16 14 10 12 09 11
08 07 06 05 04 03 02 01 13 15 14 16 09 11 10 12
09 10 11 12 13 15 14 16 02 01 04 03 06 08 05 07
10 09 12 11 15 13 16 14 01 02 03 04 05 07 06 08
11 12 09 10 14 16 13 15 04 03 02 01 08 06 07 05
12 11 10 09 16 14 15 13 03 04 01 02 07 05 08 06
13 15 14 16 09 10 11 12 07 08 05 06 03 01 04 02
14 16 13 15 11 12 09 10 05 06 07 08 01 03 02 04
15 13 16 14 10 09 12 11 08 07 06 05 04 02 03 01
16 14 15 13 12 11 10 09 06 05 08 07 02 04 01 03""")
closes(G16_3)

{frozenset({1}),
 frozenset({1, 4}),
 frozenset({1, 7}),
 frozenset({1, 3, 13, 14}),
 frozenset({1, 2}),
 frozenset({1, 3}),
 frozenset({1, 3, 15, 16}),
 frozenset({1, 5}),
 frozenset({1, 8}),
 frozenset({1, 2, 11, 12}),
 frozenset({1, 6}),
 frozenset({1, 2, 9, 10})}

## G16_13 = (Z4 x Z2) x| Z2 !!!

In [139]:
G16_13 = Group("""01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16
02 01 04 03 06 05 08 07 10 09 12 11 14 13 16 15
03 04 01 02 10 09 12 11 06 05 08 07 15 16 13 14
04 03 02 01 09 10 11 12 05 06 07 08 16 15 14 13
05 06 09 10 01 02 14 13 03 04 16 15 08 07 12 11
06 05 10 09 02 01 13 14 04 03 15 16 07 08 11 12
07 08 11 12 13 14 01 02 15 16 03 04 05 06 09 10
08 07 12 11 14 13 02 01 16 15 04 03 06 05 10 09
09 10 05 06 04 03 15 16 02 01 13 14 12 11 08 07
10 09 06 05 03 04 16 15 01 02 14 13 11 12 07 08
11 12 07 08 16 15 04 03 14 13 02 01 09 10 05 06
12 11 08 07 15 16 03 04 13 14 01 02 10 09 06 05
13 14 15 16 07 08 06 05 11 12 10 09 02 01 04 03
14 13 16 15 08 07 05 06 12 11 09 10 01 02 03 04
15 16 13 14 12 11 09 10 08 07 05 06 04 03 02 01
16 15 14 13 11 12 10 09 07 08 06 05 03 04 01 02""")
closes(G16_13)

{frozenset({1}),
 frozenset({1, 4}),
 frozenset({1, 2, 15, 16}),
 frozenset({1, 7}),
 frozenset({1, 2}),
 frozenset({1, 5}),
 frozenset({1, 8}),
 frozenset({1, 2, 11, 12}),
 frozenset({1, 6}),
 frozenset({1, 2, 9, 10}),
 frozenset({1, 3}),
 frozenset({1, 2, 13, 14})}

## G16_8 = QD16 !!!

In [140]:
G16_8 = Group("""01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16
02 01 04 03 06 05 08 07 10 09 13 14 11 12 16 15
03 04 01 02 08 07 06 05 14 12 16 10 15 09 13 11
04 03 02 01 07 08 05 06 12 14 15 09 16 10 11 13
05 06 07 08 01 02 03 04 13 11 10 15 09 16 12 14
06 05 08 07 02 01 04 03 11 13 09 16 10 15 14 12
07 08 05 06 04 03 02 01 16 15 14 11 12 13 09 10
08 07 06 05 03 04 01 02 15 16 12 13 14 11 10 09
09 10 11 13 12 14 15 16 02 01 04 06 03 05 08 07
10 09 13 11 14 12 16 15 01 02 03 05 04 06 07 08
11 13 09 10 16 15 14 12 05 06 07 01 08 02 03 04
12 14 15 16 09 10 11 13 03 04 01 08 02 07 06 05
13 11 10 09 15 16 12 14 06 05 08 02 07 01 04 03
14 12 16 15 10 09 13 11 04 03 02 07 01 08 05 06
15 16 12 14 13 11 10 09 07 08 05 04 06 03 02 01
16 15 14 12 11 13 09 10 08 07 06 03 05 04 01 02""")
closes(G16_8)

{frozenset({1}),
 frozenset({1, 4}),
 frozenset({1, 2}),
 frozenset({1, 2, 15, 16}),
 frozenset({1, 5}),
 frozenset({1, 6}),
 frozenset({1, 2, 9, 10}),
 frozenset({1, 2, 7, 8}),
 frozenset({1, 3}),
 frozenset({1, 2, 7, 8, 11, 12, 13, 14})}

In [141]:
# frozenset({1}),
# frozenset({1, 4}),                        4
# frozenset({1, 5}),                        5
# frozenset({1, 6}),                        6
# frozenset({1, 3}),                        3
# frozenset({1, 2, 15, 16}),                15       G4_1
# frozenset({1, 2, 9, 10}),                 9        G4_1
# frozenset({1, 2, 7, 8, 11, 12, 13, 14})}  11       G8_1
#           {1, 2}                          2
# frozenset({1, 2, 7, 8}),                  7        G4_1
# 
#           {1, 2, 3, 4}                    4,3      G4_0
#           {1, 2, 5, 6}                    5,6      G4_0
#           {1, 2, 3, 4, 5, 6, 7, 8}        3|4,5|6  G8_3
#           {1, 2, 7, 8, 9, 10, 15, 16}     15,9     G8_4


In [142]:
close(G16_8,{4,5})

{1, 2, 3, 4, 5, 6, 7, 8}

In [143]:
close(G16_8,{4,6})

{1, 2, 3, 4, 5, 6, 7, 8}

In [144]:
close(G16_8,{5,3})

{1, 2, 3, 4, 5, 6, 7, 8}

In [145]:
close(G16_8,{6,3})

{1, 2, 3, 4, 5, 6, 7, 8}

In [146]:
close(G16_8,{4,3})

{1, 2, 3, 4}

In [147]:
close(G16_8,{5,6})

{1, 2, 5, 6}

In [148]:
close(G16_8,{15,9})

{1, 2, 7, 8, 9, 10, 15, 16}

In [149]:
close(G16_8,{2,6})

{1, 2, 5, 6}

In [150]:
findMorphism(G2,G16_8,False)

{(1, 2), (1, 3), (1, 4), (1, 5), (1, 6)}

In [151]:
findMorphism(G4_1,G16_8,False)

{(1, 2, 7, 8),
 (1, 2, 8, 7),
 (1, 2, 9, 10),
 (1, 2, 10, 9),
 (1, 2, 15, 16),
 (1, 2, 16, 15)}

In [152]:
findMorphism(G4_0,G16_8,False)

{(1, 2, 3, 4),
 (1, 2, 4, 3),
 (1, 2, 5, 6),
 (1, 2, 6, 5),
 (1, 3, 2, 4),
 (1, 3, 4, 2),
 (1, 4, 2, 3),
 (1, 4, 3, 2),
 (1, 5, 2, 6),
 (1, 5, 6, 2),
 (1, 6, 2, 5),
 (1, 6, 5, 2)}

In [153]:
findMorphism(G8_1,G16_8,False)

{(1, 2, 7, 8, 12, 11, 14, 13),
 (1, 2, 7, 8, 14, 13, 12, 11),
 (1, 2, 8, 7, 11, 12, 13, 14),
 (1, 2, 8, 7, 13, 14, 11, 12)}

In [154]:
findMorphism(G8_3,G16_8,False)

{(1, 2, 3, 4, 5, 6, 7, 8),
 (1, 2, 3, 4, 6, 5, 8, 7),
 (1, 2, 4, 3, 5, 6, 8, 7),
 (1, 2, 4, 3, 6, 5, 7, 8),
 (1, 2, 5, 6, 3, 4, 8, 7),
 (1, 2, 5, 6, 4, 3, 7, 8),
 (1, 2, 6, 5, 3, 4, 7, 8),
 (1, 2, 6, 5, 4, 3, 8, 7)}

In [155]:
findMorphism(G8_4,G16_8,False)

{(1, 2, 7, 8, 9, 10, 15, 16),
 (1, 2, 7, 8, 10, 9, 16, 15),
 (1, 2, 7, 8, 15, 16, 10, 9),
 (1, 2, 7, 8, 16, 15, 9, 10),
 (1, 2, 8, 7, 9, 10, 16, 15),
 (1, 2, 8, 7, 10, 9, 15, 16),
 (1, 2, 8, 7, 15, 16, 9, 10),
 (1, 2, 8, 7, 16, 15, 10, 9),
 (1, 2, 9, 10, 7, 8, 16, 15),
 (1, 2, 9, 10, 8, 7, 15, 16),
 (1, 2, 9, 10, 15, 16, 7, 8),
 (1, 2, 9, 10, 16, 15, 8, 7),
 (1, 2, 10, 9, 7, 8, 15, 16),
 (1, 2, 10, 9, 8, 7, 16, 15),
 (1, 2, 10, 9, 15, 16, 8, 7),
 (1, 2, 10, 9, 16, 15, 7, 8),
 (1, 2, 15, 16, 7, 8, 9, 10),
 (1, 2, 15, 16, 8, 7, 10, 9),
 (1, 2, 15, 16, 9, 10, 8, 7),
 (1, 2, 15, 16, 10, 9, 7, 8),
 (1, 2, 16, 15, 7, 8, 10, 9),
 (1, 2, 16, 15, 8, 7, 9, 10),
 (1, 2, 16, 15, 9, 10, 7, 8),
 (1, 2, 16, 15, 10, 9, 8, 7)}

## G16_12 = Dic2 x Z2 !!!

In [156]:
G16_12 = Group("""01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16
02 01 04 03 06 05 08 07 10 09 12 11 14 13 16 15
03 04 01 02 07 08 05 06 11 12 09 10 15 16 13 14
04 03 02 01 08 07 06 05 12 11 10 09 16 15 14 13
05 06 07 08 02 01 04 03 14 13 16 15 09 10 11 12
06 05 08 07 01 02 03 04 13 14 15 16 10 09 12 11
07 08 05 06 04 03 02 01 16 15 14 13 11 12 09 10
08 07 06 05 03 04 01 02 15 16 13 14 12 11 10 09
09 10 11 12 13 14 15 16 02 01 04 03 06 05 08 07
10 09 12 11 14 13 16 15 01 02 03 04 05 06 07 08
11 12 09 10 15 16 13 14 04 03 02 01 08 07 06 05
12 11 10 09 16 15 14 13 03 04 01 02 07 08 05 06
13 14 15 16 10 09 12 11 05 06 07 08 02 01 04 03
14 13 16 15 09 10 11 12 06 05 08 07 01 02 03 04
15 16 13 14 12 11 10 09 07 08 05 06 04 03 02 01
16 15 14 13 11 12 09 10 08 07 06 05 03 04 01 02""")
closes(G16_12)

{frozenset({1}),
 frozenset({1, 4}),
 frozenset({1, 2}),
 frozenset({1, 2, 15, 16}),
 frozenset({1, 2, 5, 6}),
 frozenset({1, 2, 11, 12}),
 frozenset({1, 2, 9, 10}),
 frozenset({1, 2, 7, 8}),
 frozenset({1, 3}),
 frozenset({1, 2, 13, 14})}

## G16_4 = Z4 x| Z4 !!!

In [157]:
G12_2 = Group("""01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16
02 01 04 03 06 05 08 07 10 09 12 11 15 16 13 14
03 04 01 02 07 08 05 06 11 12 09 10 14 13 16 15
04 03 02 01 08 07 06 05 12 11 10 09 16 15 14 13
05 06 07 08 02 01 04 03 14 16 13 15 12 10 11 09
06 05 08 07 01 02 03 04 16 14 15 13 11 09 12 10
07 08 05 06 04 03 02 01 13 15 14 16 10 12 09 11
08 07 06 05 03 04 01 02 15 13 16 14 09 11 10 12
09 10 11 12 13 15 14 16 02 01 04 03 06 08 05 07
10 09 12 11 15 13 16 14 01 02 03 04 05 07 06 08
11 12 09 10 14 16 13 15 04 03 02 01 08 06 07 05
12 11 10 09 16 14 15 13 03 04 01 02 07 05 08 06
13 15 14 16 10 09 12 11 08 07 06 05 03 01 04 02
14 16 13 15 12 11 10 09 06 05 08 07 01 03 02 04
15 13 16 14 09 10 11 12 07 08 05 06 04 02 03 01
16 14 15 13 11 12 09 10 05 06 07 08 02 04 01 03""")
closes(G12_2)

{frozenset({1}),
 frozenset({1, 4}),
 frozenset({1, 3, 13, 14}),
 frozenset({1, 2}),
 frozenset({1, 2, 5, 6}),
 frozenset({1, 3, 15, 16}),
 frozenset({1, 2, 11, 12}),
 frozenset({1, 2, 9, 10}),
 frozenset({1, 2, 7, 8}),
 frozenset({1, 3})}

## G16_2 = Z4 x Z4

In [158]:
G16_2 = Group("""01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16
02 01 04 03 06 05 08 07 11 12 09 10 15 16 13 14
03 04 01 02 07 08 05 06 10 09 12 11 16 15 14 13
04 03 02 01 08 07 06 05 12 11 10 09 14 13 16 15
05 06 07 08 02 01 04 03 13 16 15 14 11 10 09 12
06 05 08 07 01 02 03 04 15 14 13 16 09 12 11 10
07 08 05 06 04 03 02 01 16 13 14 15 12 09 10 11
08 07 06 05 03 04 01 02 14 15 16 13 10 11 12 09
09 11 10 12 13 15 16 14 03 01 04 02 07 06 08 05
10 12 09 11 16 14 13 15 01 03 02 04 05 08 06 07
11 09 12 10 15 13 14 16 04 02 03 01 08 05 07 06
12 10 11 09 14 16 15 13 02 04 01 03 06 07 05 08
13 15 16 14 11 09 12 10 07 05 08 06 04 01 03 02
14 16 15 13 10 12 09 11 06 08 05 07 01 04 02 03
15 13 14 16 09 11 10 12 08 06 07 05 03 02 04 01
16 14 13 15 12 10 11 09 05 07 06 08 02 03 01 04""")
closes(G16_2)

{frozenset({1}),
 frozenset({1, 4}),
 frozenset({1, 2}),
 frozenset({1, 2, 5, 6}),
 frozenset({1, 3}),
 frozenset({1, 3, 9, 10}),
 frozenset({1, 3, 11, 12}),
 frozenset({1, 4, 13, 14}),
 frozenset({1, 2, 7, 8}),
 frozenset({1, 4, 15, 16})}

## G16_5 = Z8 x Z2

In [159]:
G16_5 = Group("""01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16
02 01 04 03 06 05 08 07 11 12 09 10 15 16 13 14
03 04 01 02 07 08 05 06 13 14 15 16 09 10 11 12
04 03 02 01 08 07 06 05 15 16 13 14 11 12 09 10
05 06 07 08 02 01 04 03 10 11 12 09 14 15 16 13
06 05 08 07 01 02 03 04 12 09 10 11 16 13 14 15
07 08 05 06 04 03 02 01 14 15 16 13 10 11 12 09
08 07 06 05 03 04 01 02 16 13 14 15 12 09 10 11
09 11 13 15 10 12 14 16 06 01 05 02 08 03 07 04
10 12 14 16 11 09 15 13 01 05 02 06 03 07 04 08
11 09 15 13 12 10 16 14 05 02 06 01 07 04 08 03
12 10 16 14 09 11 13 15 02 06 01 05 04 08 03 07
13 15 09 11 14 16 10 12 08 03 07 04 06 01 05 02
14 16 10 12 15 13 11 09 03 07 04 08 01 05 02 06
15 13 11 09 16 14 12 10 07 04 08 03 05 02 06 01
16 14 12 10 13 15 09 11 04 08 03 07 02 06 01 05""")
closes(G16_5)

{frozenset({1}),
 frozenset({1, 4}),
 frozenset({1, 2}),
 frozenset({1, 2, 5, 6}),
 frozenset({1, 2, 5, 6, 9, 10, 11, 12}),
 frozenset({1, 2, 5, 6, 13, 14, 15, 16}),
 frozenset({1, 2, 7, 8}),
 frozenset({1, 3})}

## G16_6 = Z8 x| Z5 !!!

In [160]:
G16_6 = Group("""01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16
02 01 04 03 06 05 08 07 11 12 09 10 15 16 13 14
03 04 01 02 07 08 05 06 15 14 13 16 11 10 09 12
04 03 02 01 08 07 06 05 13 16 15 14 09 12 11 10
05 06 07 08 02 01 04 03 10 11 12 09 16 13 14 15
06 05 08 07 01 02 03 04 12 09 10 11 14 15 16 13
07 08 05 06 04 03 02 01 14 13 16 15 12 11 10 09
08 07 06 05 03 04 01 02 16 15 14 13 10 09 12 11
09 11 13 15 10 12 16 14 06 01 05 02 08 04 07 03
10 12 16 14 11 09 15 13 01 05 02 06 03 08 04 07
11 09 15 13 12 10 14 16 05 02 06 01 07 03 08 04
12 10 14 16 09 11 13 15 02 06 01 05 04 07 03 08
13 15 09 11 16 14 10 12 07 04 08 03 05 01 06 02
14 16 12 10 13 15 09 11 03 07 04 08 01 06 02 05
15 13 11 09 14 16 12 10 08 03 07 04 06 02 05 01
16 14 10 12 15 13 11 09 04 08 03 07 02 05 01 06""")
closes(G16_6)

{frozenset({1}),
 frozenset({1, 4}),
 frozenset({1, 2}),
 frozenset({1, 2, 5, 6}),
 frozenset({1, 2, 5, 6, 9, 10, 11, 12}),
 frozenset({1, 2, 5, 6, 13, 14, 15, 16}),
 frozenset({1, 2, 7, 8}),
 frozenset({1, 3})}

## G16_9 = Dic4 !!!

In [161]:
G16_9 = Group("""01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16
02 01 04 03 06 05 08 07 10 09 12 11 15 16 13 14
03 04 02 01 08 07 05 06 12 11 09 10 14 15 16 13
04 03 01 02 07 08 06 05 11 12 10 09 16 13 14 15
05 06 07 08 02 01 04 03 14 16 15 13 11 10 12 09
06 05 08 07 01 02 03 04 16 14 13 15 12 09 11 10
07 08 06 05 03 04 02 01 13 15 14 16 10 12 09 11
08 07 05 06 04 03 01 02 15 13 16 14 09 11 10 12
09 10 11 12 13 15 14 16 02 01 04 03 06 08 05 07
10 09 12 11 15 13 16 14 01 02 03 04 05 07 06 08
11 12 10 09 16 14 13 15 03 04 02 01 08 05 07 06
12 11 09 10 14 16 15 13 04 03 01 02 07 06 08 05
13 15 14 16 10 09 12 11 08 07 05 06 04 01 03 02
14 16 15 13 11 12 10 09 06 05 08 07 01 03 02 04
15 13 16 14 09 10 11 12 07 08 06 05 03 02 04 01
16 14 13 15 12 11 09 10 05 06 07 08 02 04 01 03""")
closes(G16_9)

{frozenset({1}),
 frozenset({1, 2}),
 frozenset({1, 2, 5, 6}),
 frozenset({1, 2, 3, 4, 13, 14, 15, 16}),
 frozenset({1, 2, 11, 12}),
 frozenset({1, 2, 3, 4}),
 frozenset({1, 2, 9, 10}),
 frozenset({1, 2, 7, 8})}

## G16_1 = Z16

In [162]:
G16_1 = Group("""01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16
02 01 04 03 07 08 05 06 11 12 09 10 15 16 13 14
03 04 02 01 06 07 08 05 13 16 15 14 11 10 09 12
04 03 01 02 08 05 06 07 15 14 13 16 09 12 11 10
05 07 06 08 04 01 03 02 10 15 12 13 16 11 14 09
06 08 07 05 01 03 02 04 16 09 14 11 12 15 10 13
07 05 08 06 03 02 04 01 12 13 10 15 14 09 16 11
08 06 05 07 02 04 01 03 14 11 16 09 10 13 12 15
09 11 13 15 10 16 12 14 06 01 08 02 07 04 05 03
10 12 16 14 15 09 13 11 01 05 02 07 03 08 04 06
11 09 15 13 12 14 10 16 08 02 06 01 05 03 07 04
12 10 14 16 13 11 15 09 02 07 01 05 04 06 03 08
13 15 11 09 16 12 14 10 07 03 05 04 08 01 06 02
14 16 10 12 11 15 09 13 04 08 03 06 01 07 02 05
15 13 09 11 14 10 16 12 05 04 07 03 06 02 08 01
16 14 12 10 09 13 11 15 03 06 04 08 02 05 01 07""")
closes(G16_1)

{frozenset({1}),
 frozenset({1, 2}),
 frozenset({1, 2, 3, 4}),
 frozenset({1, 2, 3, 4, 5, 6, 7, 8}),
 frozenset({1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16})}

# 17

# 13

# 13

# 13

# 13

# 13

# 13

# 13

смежные классы
порождающие
порядки элементов

вернуть порядок перебора
сделать своё deepcopy

правила вычёркивания

добаить .name - имя группы, и её другие названия/разложения
в конструкторе Group(параметры, name=name)

----

сделать аналог closes для одного элемента и назвать cycles, и сделать map: цикл -> его генераторы
    цикл представлять не множеством а туплом
find_morphysm добавить find_morphism_s, в описании написать, что это мономорфизм, посчитать сложность его поиска

проверить, чтобы эти функции были полиморфными, и требовали от группы только умножение и .inv()

----

сделать базу данных групп
? как найти группу:
	1) порядок группы
	2) набор циклов (граф циклов - сложно)
	3) построить изоморфизм в каждого кандидата
*        понять, чем отличаются разные группы с одинаковыми графами
*        поискать группы с одинаковыми наборами циклов, но разными графами

----

* написать функцию, которая ищет ВСЕ подгруппы, попытаться доказать, что она ище ВСЕ подгруппы  
? как найти все подгруппы?  
у каждого элемента найти цикл, одинаковые циклы объединить, оставить у каждого по одному генератору, подциклы отложить в сторону  
нагенерить новые группы из пар генераторов (различных), одинаковые объединить  
нагенерить новые группы из троек (различных): всех пар предыдущего шага и генераторов первого шага, одинаковые объединить  
...  
поискать мономорфизмы из нециклических простых групп  
добавить отложенные в сторону циклы  

сделать функцию, которая для подгруппы будет генерировать правый и левый смежный классы

* убедиться, что если подгруппа ненормальная, то ни как нельзя построить группу на смежных классах

---

сделать производные классы:
SubGroup(G,subset)
    данные: subset
    поведение: определяется G
FactorGroup(G,subset)
    данные: bimap: N <-> смежный класс
    поведение: определяется G
и полиморфный класс:
Permutation(список генераторов)
    перестановка - тупл длины N или тупл туплов-циклов
    
сделать функции, которые создают группы:
Aut(G)
Zenter(G)
Int(G)

* убедиться, что Int(G) всегда нормальная подгруппа Aut(G)

---

* * понять полупрямое произведение

сделать красивую печать:
.prorder // print order - другой порядок элементов привыводе таблицы
.renames - другие имена элементов

* * придумать эффективный алгоритм поиска групп данного порядка и нагенерировать их до 95го порядка

---
* поискать и поклассифицировать погдгруппы групп Sn, An
* поискать у групп минимальные надгруппы Sn

---
* поискать у групп копредставления (задания)
-------------------------------------------
геометрии Лобачевского
          проективная (двойственность)
          
дифф Гем Фоменко
    пр. Лежандра -> термодинамика
    ур-е Гамильтона-Якоби -> теормех, квантмех
    метрика Римана -> ОТО
... группы Ли

-------------------------------------------
? можно ли любую дискретную группу представить в виде матриц с целыми коэф-тами (конечных или бесконечных)?
? можно ли рациональные числа по сложению представить в виде матриц?
